In [1]:
%pip install opencv-python
%pip install timm  # For EfficientNetV2 models
%pip install gdown

# Basic imports
import os
import cv2
import pandas as pd
import random
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from tqdm import tqdm

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import timm  # For EfficientNetV2 models

# Image processing
from PIL import Image


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\dprih\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\dprih\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\dprih\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'timm'

# Dataset Preparation

Clone the dataset from the repository using Git.

In [ ]:
# # Normal tidak di white balance
# !gdown https://drive.google.com/uc?id=1JUbef73bC02xEeGZveCBQ0HOSM-Q9zGU
# !gdown https://drive.google.com/uc?id=1P_ry-xorev301eMAr10a_Cht9Si7BDC9
# !gdown https://drive.google.com/uc?id=1qmf7YizdXe2XhPIsA43onUVn-MYYYDVw
# !unzip "Data_Asli.zip" -d dataset_asli

# # Dilakukan old contrast enhancement
# !gdown https://drive.google.com/uc?id=1_qa3nGhPM7_LnzBRFm1XVZbg-ypYS8M6
# !gdown https://drive.google.com/uc?id=1XTFp5v0UJe6wy3II4hwqWvV8exkIJSpp
# !gdown https://drive.google.com/uc?id=1cMWFtBrePO08hjHOdYZE7PRZV-zZfK52
# 10.000 data ASLI GAN RATIO
# !gdown https://drive.google.com/uc?id=1s7Cu9chsRrI30Qkl5u13VuqPwWysfNsr

# Dilakukan NEW contrast enhancement FILTER
!gdown https://drive.google.com/uc?id=1BHrO8ZM08NqGFBnZjHDlmSYFYg7f8Z1U
!gdown https://drive.google.com/uc?id=1x6JVhPgO2Fq9yNGK57iD4RPflKdpM3vI
!gdown https://drive.google.com/uc?id=1MfKxrt1YzIRVdlUxW881hrEBIx335gUE

In [ ]:
# !unzip "Dataset_Asli.zip" -d dataset_asli
# !unzip "Dataset_Asli_GAN.zip" -d dataset_asli_gan
# !unzip "Dataset_Asli_GAN_Ratio.zip" -d dataset_asli_gan_ratio

!unzip "Dataset_Asli_Enhanced.zip" -d dataset_asli_enhanced
!unzip "Dataset_Asli_GAN_Enhanced.zip" -d dataset_asli_gan_enhanced
!unzip "Dataset_Asli_GAN_Ratio_Enhanced.zip" -d dataset_asli_gan_ratio_enhanced

In [ ]:
# Training Configuration
BATCH_SIZE = 32  # Reduced for better stability
IMAGE_SIZE = (224, 224)  # EfficientNetV2B0 default input size
NUM_CLASSES = 3  # L1, L2, L3
RANDOM_SEED = 42

# Dataset paths
# Normal tidak di white balance
# BASE_DIR = './dataset_asli/Dataset_Asli'  # Current directory where L1, L2, L3 folders are located
# BASE_DIR_GAN = './dataset_asli_gan/Dataset_Asli_GAN'  # Current directory where L1, L2, L3 folders are located
# BASE_DIR_GAN_RATIO = './dataset_asli_gan_ratio/Dataset_Asli_GAN_Ratio'  # Current directory where L1, L2, L3 folders are located

# Contrast Enhancement
BASE_DIR = './dataset_asli_enhanced/Dataset_Asli_Enhanced'  # Current directory where L1, L2, L3 folders are located
BASE_DIR_GAN = './dataset_asli_gan_enhanced/Dataset_Asli_GAN_Enhanced'  # Current directory where L1, L2, L3 folders are located
BASE_DIR_GAN_RATIO = './dataset_asli_gan_ratio_enhanced/Dataset_Asli_GAN_Ratio_Enhanced'  # Current directory where L1, L2, L3 folders are located

CLASSES = ['L1', 'L2', 'L3']

# Training parameters
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-4
NUM_EPOCHS = 100
EARLY_STOPPING_PATIENCE_5 = 5
EARLY_STOPPING_PATIENCE_10 = 10
EARLY_STOPPING_PATIENCE_15 = 15
EARLY_STOPPING_PATIENCE_20 = 20
EARLY_STOPPING_PATIENCE_NONE = None


# Device configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torch.utils.data import WeightedRandomSampler
import numpy as np
import os # Pastikan os diimpor

def create_weighted_dataloader(dataset, batch_size, num_classes, num_workers=0, original_data_weight_multiplier=2.0):
    """
    Membuat DataLoader dengan WeightedRandomSampler untuk menangani ketidakseimbangan kelas
    dan memberikan bobot berbeda pada data asli vs. data GAN, berdasarkan awalan nama file "seed".

    Args:
        dataset (torch.utils.data.Dataset): Dataset untuk dibuat DataLoader-nya.
        batch_size (int): Ukuran batch.
        num_classes (int): Jumlah total kelas.
        num_workers (int): Jumlah worker untuk loading data.
        original_data_weight_multiplier (float): Multiplier bobot untuk data asli. Default 2.0.

    Returns:
        torch.utils.data.DataLoader: DataLoader dengan WeightedRandomSampler.
    """
    if not hasattr(dataset, 'labels') or not dataset.labels:
        print(f"Warning: Dataset {dataset.split} has no 'labels' attribute or it's empty. Cannot create weighted sampler. Using shuffle=True.")
        return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

    labels = dataset.labels
    image_paths = dataset.data # Asumsi dataset.data adalah list of image paths

    # Hitung jumlah setiap kelas
    class_counts = np.bincount(labels, minlength=num_classes)
    class_counts = np.maximum(class_counts, 1)
    class_weights_base = 1.0 / class_counts

    sample_weights = []
    # Definisi awalan nama file untuk gambar GAN
    gan_file_prefix = "seed"

    for i, label in enumerate(labels):
        weight = class_weights_base[label]

        img_filename = os.path.basename(image_paths[i]) # Ambil nama file dari jalur lengkap

        # Deteksi apakah ini gambar GAN berdasarkan awalan nama file
        is_gan_image = img_filename.startswith(gan_file_prefix)

        if not is_gan_image: # Jika ini adalah gambar ASLI (tidak diawali "seed")
            weight *= original_data_weight_multiplier # Beri bobot ekstra pada data asli

        sample_weights.append(weight)

    sample_weights = torch.DoubleTensor(sample_weights)

    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        sampler=sampler,
        num_workers=num_workers
    )
    return dataloader

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(RANDOM_SEED)

# Definisikan model dummy untuk EfficientNetV2-B0
dummy_model_effnetv2 = timm.create_model('tf_efficientnetv2_s.in21k_ft_in1k', pretrained=True)
# Definisikan model dummy untuk ResNet50
dummy_model_resnet50 = timm.create_model('resnet50', pretrained=True)

class Dataset(Dataset):
    CLASS_NAMES = ['L1', 'L2', 'L3']
    CLASS_TO_LABEL = {name: idx for idx, name in enumerate(CLASS_NAMES)}

    def __init__(self, root_dir=None, split='train', print_info=True, model_name_for_transforms=None):
        self.root_dir = root_dir
        self.split = split
        self.data = []
        self.labels = []
        self.image_sizes = {}

        # Ini yang baru: mendapatkan transform sesuai nama model
        if model_name_for_transforms == 'efficientnetv2-training':
            data_config = timm.data.resolve_model_data_config(dummy_model_effnetv2)
            self.transform = timm.data.create_transform(**data_config, is_training=True)
        elif model_name_for_transforms == 'efficientnetv2':
            data_config = timm.data.resolve_model_data_config(dummy_model_effnetv2)
            self.transform = timm.data.create_transform(**data_config, is_training=False)
        elif model_name_for_transforms == 'resnet50-training':
            data_config = timm.data.resolve_model_data_config(dummy_model_resnet50)
            self.transform = timm.data.create_transform(**data_config, is_training=True)
        elif model_name_for_transforms == 'resnet50':
            data_config = timm.data.resolve_model_data_config(dummy_model_resnet50)
            self.transform = timm.data.create_transform(**data_config, is_training=False)
        elif model_name_for_transforms == 'None':
            print("Pretrained False")
            # self.transform = transforms.Compose([
            #     transforms.Resize((256, 256)),
            #     transforms.ToTensor(),
            #     # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            # ])
            self.transform = transforms.Compose([
                transforms.Resize((256, 256)), # Ukuran yang konsisten adalah penting
                # transforms.RandomHorizontalFlip(),  # Flip horizontal
                # transforms.RandomRotation(degrees=15), # Rotasi acak hingga 15 derajat
                # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Perubahan warna
                # transforms.RandomRotation(degrees=(-45, 45)),
                # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                # transforms.RandomAffine(degrees=(-10, 10), translate=(0.1, 0.1), scale=(0.9, 1.1), shear=(-5, 5)),
                # transforms.GaussianBlur(kernel_size=(3, 3)), # Pertimbangkan jika dataset Anda cenderung buram
                # transforms.RandomPerspective(distortion_scale=0.2, p=0.2), # Gunakan dengan hati-hati
                transforms.ToTensor(), # Ubah ke tensor
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # Normalisasi setelah ToTensor
            ])
        else:
            # Fallback transform jika model_name_for_transforms tidak dikenali
            print("Warning: Using default transform. Specify 'efficientnetv2_b0' or 'resnet50' for model_name_for_transforms.")
            self.transform = transforms.Compose([
                transforms.Resize((256, 256)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # Normalisasi setelah ToTensor
            ])

        if root_dir:
            start = time.time()
            self._load_images()
            if print_info:
                self._print_info(round(time.time() - start, 2))

    def _load_images(self):
        for class_name, label in self.CLASS_TO_LABEL.items():
            # Construct path to the specific split folder (train/val/test) for each class
            class_dir = os.path.join(self.root_dir, class_name, self.split)
            if not os.path.exists(class_dir):
                continue
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                try:
                    with Image.open(img_path) as img:
                        img = img.convert("RGB")
                        self.image_sizes[img.size] = self.image_sizes.get(img.size, 0) + 1
                    self.data.append(img_path)
                    self.labels.append(label)
                except Exception as e:
                    print(f"Failed to load image {img_path}: {e}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        label = self.labels[idx]

# Load and transform image
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)


        # image = Image.open(img_path).convert("RGB")
        return image, label

    def to_dataframe(self):
        return pd.DataFrame({
            'image_path': self.data,
            'label': self.labels
        })

    def _print_info(self, elapsed):
        print("========== Dataset Summary ==========")
        print(f"Split: {self.split}")
        print(f"Total images       : {len(self.data)}")
        print(f"Number of classes  : {len(self.CLASS_NAMES)}")

        # Images per class table
        print("\n--- Images per Class ---")
        print(f"{'Label':<7} | {'Class Name':<25} | {'Image Count':>12}")
        print("-" * 50)
        class_counts = {name: 0 for name in self.CLASS_NAMES}
        for label in self.labels:
            class_name = self.CLASS_NAMES[label]
            class_counts[class_name] += 1
        for label, class_name in enumerate(self.CLASS_NAMES):
            count = class_counts[class_name]
            print(f"{label:<7} | {class_name:<25} | {count:>12}")

        # Unique image sizes
        print("\n--- Unique Image Sizes ---")
        print(f"{'Image Size (WxH)':<25} | {'Count':>12}")
        print("-" * 40)
        for size, count in sorted(self.image_sizes.items(), key=lambda x: -x[1]):
            size_str = f"{size[0]}x{size[1]}"
            print(f"{size_str:<25} | {count:>12}")

        print(f"\nLoaded in {elapsed:.2f} seconds ✅")

# Define image transformations
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),  # EfficientNetV2B0 default input size
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
# ])

# Initialize datasets for each split
print('--------------------------------------------------------------------')
print('--------------------------- Pure Dataset ---------------------------')
print('--------------------------------------------------------------------')
print('---------------------------Data Asli---------------------------')
train_dataset_asli = Dataset(BASE_DIR, split='train',  model_name_for_transforms='None')
val_dataset_asli = Dataset(BASE_DIR, split='val',   model_name_for_transforms=None)
test_dataset_asli = Dataset(BASE_DIR, split='test',   model_name_for_transforms=None)
print('---------------------------Data Asli + GAN---------------------------')
train_dataset_asli_gan = Dataset(BASE_DIR_GAN, split='train',  model_name_for_transforms='None')
val_dataset_asli_gan = Dataset(BASE_DIR_GAN, split='val',   model_name_for_transforms=None)
test_dataset_asli_gan = Dataset(BASE_DIR_GAN, split='test',   model_name_for_transforms=None)
print('---------------------------Data Asli + GAN + Ratio---------------------------')
train_dataset_asli_gan_ratio = Dataset(BASE_DIR_GAN_RATIO, split='train',  model_name_for_transforms='None')
val_dataset_asli_gan_ratio = Dataset(BASE_DIR_GAN_RATIO, split='val',   model_name_for_transforms=None)
test_dataset_asli_gan_ratio = Dataset(BASE_DIR_GAN_RATIO, split='test',   model_name_for_transforms=None)

print('-----------------------------------------------------------------------------------')
print('--------------------------- (Resnet) Pretrained Dataset ---------------------------')
print('-----------------------------------------------------------------------------------')
print('---------------------------Data Asli---------------------------')
train_dataset_asli_pretrained_resnet = Dataset(BASE_DIR, split='train',  model_name_for_transforms='resnet50-training')
val_dataset_asli_pretrained_resnet= Dataset(BASE_DIR, split='val',   model_name_for_transforms='resnet50')
test_dataset_asli_pretrained_resnet = Dataset(BASE_DIR, split='test',   model_name_for_transforms='resnet50')
print('---------------------------Data Asli + GAN---------------------------')
train_dataset_asli_gan_pretrained_resnet = Dataset(BASE_DIR_GAN, split='train',  model_name_for_transforms='resnet50-training')
val_dataset_asli_gan_pretrained_resnet = Dataset(BASE_DIR_GAN, split='val',   model_name_for_transforms='resnet50')
test_dataset_asli_gan_pretrained_resnet = Dataset(BASE_DIR_GAN, split='test',   model_name_for_transforms='resnet50')
print('---------------------------Data Asli + GAN + Ratio---------------------------')
train_dataset_asli_gan_ratio_pretrained_resnet = Dataset(BASE_DIR_GAN_RATIO, split='train',  model_name_for_transforms='resnet50-training')
val_dataset_asli_gan_ratio_pretrained_resnet = Dataset(BASE_DIR_GAN_RATIO, split='val',   model_name_for_transforms='resnet50')
test_dataset_asli_gan_ratio_pretrained_resnet = Dataset(BASE_DIR_GAN_RATIO, split='test',   model_name_for_transforms='resnet50')

print('-----------------------------------------------------------------------------------')
print('--------------------------- (EfficientNetV2) Pretrained Dataset ---------------------------')
print('-----------------------------------------------------------------------------------')
print('---------------------------Data Asli---------------------------')
train_dataset_asli_pretrained_efficientnet = Dataset(BASE_DIR, split='train',  model_name_for_transforms='efficientnetv2-training')
val_dataset_asli_pretrained_efficientnet= Dataset(BASE_DIR, split='val',   model_name_for_transforms='efficientnetv2')
test_dataset_asli_pretrained_efficientnet = Dataset(BASE_DIR, split='test',   model_name_for_transforms='efficientnetv2')
print('---------------------------Data Asli + GAN---------------------------')
train_dataset_asli_gan_pretrained_efficientnet = Dataset(BASE_DIR_GAN, split='train',  model_name_for_transforms='efficientnetv2-training')
val_dataset_asli_gan_pretrained_efficientnet = Dataset(BASE_DIR_GAN, split='val',   model_name_for_transforms='efficientnetv2')
test_dataset_asli_gan_pretrained_efficientnet = Dataset(BASE_DIR_GAN, split='test',   model_name_for_transforms='efficientnetv2')
print('---------------------------Data Asli + GAN + Ratio---------------------------')
train_dataset_asli_gan_ratio_pretrained_efficientnet = Dataset(BASE_DIR_GAN_RATIO, split='train',  model_name_for_transforms='efficientnetv2-training')
val_dataset_asli_gan_ratio_pretrained_efficientnet = Dataset(BASE_DIR_GAN_RATIO, split='val',   model_name_for_transforms='efficientnetv2')
test_dataset_asli_gan_ratio_pretrained_efficientnet = Dataset(BASE_DIR_GAN_RATIO, split='test',   model_name_for_transforms='efficientnetv2')

# Create data loaders
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
print('---------------------------------------------------------------------------')
print('--------------------------- Pure Dataset Loader ---------------------------')
print('---------------------------------------------------------------------------')
# print('---------------------------Data Asli---------------------------')
# train_loader_asli = create_weighted_dataloader(train_dataset_asli, BATCH_SIZE, NUM_CLASSES, num_workers=0)
train_loader_asli = DataLoader(train_dataset_asli, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader_asli = DataLoader(val_dataset_asli, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader_asli = DataLoader(test_dataset_asli, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
# print('---------------------------Data Asli + GAN---------------------------')
# train_loader_asli_gan = create_weighted_dataloader(train_dataset_asli_gan, BATCH_SIZE, NUM_CLASSES, num_workers=0)
train_loader_asli_gan = DataLoader(train_dataset_asli_gan, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader_asli_gan = DataLoader(val_dataset_asli_gan, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader_asli_gan = DataLoader(test_dataset_asli_gan, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
# print('---------------------------Data Asli + GAN + Ratio---------------------------')
# train_loader_asli_gan_ratio = create_weighted_dataloader(train_dataset_asli_gan_ratio, BATCH_SIZE, NUM_CLASSES, num_workers=0)
train_loader_asli_gan_ratio = DataLoader(train_dataset_asli_gan_ratio, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader_asli_gan_ratio = DataLoader(val_dataset_asli_gan_ratio, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader_asli_gan_ratio = DataLoader(test_dataset_asli_gan_ratio, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print('-----------------------------------------------------------------------------------')
print('--------------------------- (Resnet) Pretrained Dataset ---------------------------')
print('-----------------------------------------------------------------------------------')

# print('---------------------------Data Asli---------------------------')
# train_loader_asli_pretrained_resnet = create_weighted_dataloader(train_dataset_asli_pretrained_resnet, BATCH_SIZE, NUM_CLASSES, num_workers=0)
train_loader_asli_pretrained_resnet = DataLoader(train_dataset_asli_pretrained_resnet, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader_asli_pretrained_resnet = DataLoader(val_dataset_asli_pretrained_resnet, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader_asli_pretrained_resnet = DataLoader(test_dataset_asli_pretrained_resnet, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
# print('---------------------------Data Asli + GAN---------------------------')
# train_loader_asli_gan_pretrained_resnet = create_weighted_dataloader(train_dataset_asli_gan_pretrained_resnet, BATCH_SIZE, NUM_CLASSES, num_workers=0)
train_loader_asli_gan_pretrained_resnet = DataLoader(train_dataset_asli_gan_pretrained_resnet, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader_asli_gan_pretrained_resnet = DataLoader(val_dataset_asli_gan_pretrained_resnet, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader_asli_gan_pretrained_resnet = DataLoader(test_dataset_asli_gan_pretrained_resnet, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
# print('---------------------------Data Asli + GAN + Ratio---------------------------')
# train_loader_asli_gan_ratio_pretrained_resnet = create_weighted_dataloader(train_dataset_asli_gan_ratio_pretrained_resnet, BATCH_SIZE, NUM_CLASSES, num_workers=0)
train_loader_asli_gan_ratio_pretrained_resnet = DataLoader(train_dataset_asli_gan_ratio_pretrained_resnet, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader_asli_gan_ratio_pretrained_resnet = DataLoader(val_dataset_asli_gan_ratio_pretrained_resnet, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader_asli_gan_ratio_pretrained_resnet = DataLoader(test_dataset_asli_gan_ratio_pretrained_resnet, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print('----------------------------------------------------------------------------------------------')
print('--------------------------- (EfficientNetV2_B0) Pretrained Dataset ---------------------------')
print('----------------------------------------------------------------------------------------------')

# print('---------------------------Data Asli---------------------------')
# train_loader_asli_pretrained_efficientnet = create_weighted_dataloader(train_dataset_asli_pretrained_efficientnet, BATCH_SIZE, NUM_CLASSES, num_workers=0)
train_loader_asli_pretrained_efficientnet = DataLoader(train_dataset_asli_pretrained_efficientnet, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader_asli_pretrained_efficientnet = DataLoader(val_dataset_asli_pretrained_efficientnet, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader_asli_pretrained_efficientnet = DataLoader(test_dataset_asli_pretrained_efficientnet, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
# print('---------------------------Data Asli + GAN---------------------------')
# train_loader_asli_gan_pretrained_efficientnet = create_weighted_dataloader(train_dataset_asli_gan_pretrained_efficientnet, BATCH_SIZE, NUM_CLASSES, num_workers=0)
train_loader_asli_gan_pretrained_efficientnet = DataLoader(train_dataset_asli_gan_pretrained_efficientnet, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader_asli_gan_pretrained_efficientnet = DataLoader(val_dataset_asli_gan_pretrained_efficientnet, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader_asli_gan_pretrained_efficientnet = DataLoader(test_dataset_asli_gan_pretrained_efficientnet, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
# print('---------------------------Data Asli + GAN + Ratio---------------------------')
# train_loader_asli_gan_ratio_pretrained_efficientnet = create_weighted_dataloader(train_dataset_asli_gan_ratio_pretrained_efficientnet, BATCH_SIZE, NUM_CLASSES, num_workers=0)
train_loader_asli_gan_ratio_pretrained_efficientnet = DataLoader(train_dataset_asli_gan_ratio_pretrained_efficientnet, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader_asli_gan_ratio_pretrained_efficientnet = DataLoader(val_dataset_asli_gan_ratio_pretrained_efficientnet, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader_asli_gan_ratio_pretrained_efficientnet = DataLoader(test_dataset_asli_gan_ratio_pretrained_efficientnet, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# Visualization

In [ ]:
def plot_class_distribution(dataset, title="Image Distribution per Class", sort_by_count=True, show_percentage=True):
    """
    Generates a clean and informative bar chart of class distribution.

    Args:
        dataset (WasteDataset): The dataset to plot distribution for
        title (str): Chart title.
        sort_by_count (bool): If True, sort bars by image count descending.
        show_percentage (bool): If True, show % along with count.
    """
    # Get counts for each class
    label_counts = Counter(dataset.labels)
    total = sum(label_counts.values())

    # Convert to list of tuples (label, count)
    label_data = list(label_counts.items())

    # Sort by count if enabled
    if sort_by_count:
        label_data.sort(key=lambda x: x[1], reverse=True)
    else:
        label_data.sort(key=lambda x: x[0])  # label order

    class_names = [dataset.CLASS_NAMES[lbl] for lbl, _ in label_data]
    counts = [cnt for _, cnt in label_data]
    percentages = [cnt / total * 100 for cnt in counts]

    # Set seaborn style
    sns.set(style="whitegrid")
    palette = sns.color_palette("pastel")

    plt.figure(figsize=(10, 6))
    bars = plt.bar(class_names, counts, color=palette)

    # Add count & percentage labels
    for i, bar in enumerate(bars):
        height = bar.get_height()
        pct_text = f"{height:,}" + (f" ({percentages[i]:.1f}%)" if show_percentage else "")
        plt.text(bar.get_x() + bar.get_width() / 2, height + 5, pct_text,
                 ha='center', va='bottom', fontsize=10)

    # Labels and title
    plt.title(f"{title} - {dataset.split.capitalize()} Split", fontsize=16, weight='bold')
    plt.xlabel("Class", fontsize=12)
    plt.ylabel("Number of Images", fontsize=12)
    plt.xticks(rotation=0, fontsize=11)
    plt.yticks(fontsize=11)
    plt.grid(axis='y', linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.show()

# Plot distribution for each split
print('---------------------------Data Asli---------------------------')
print("Training Set Distribution:")
plot_class_distribution(
    train_dataset_asli,
    title="Image Distribution per Class",
    sort_by_count=False,
    show_percentage=True
)

print("\nValidation Set Distribution:")
plot_class_distribution(
    val_dataset_asli,
    title="Image Distribution per Class",
    sort_by_count=False,
    show_percentage=True
)

print("\nTest Set Distribution:")
plot_class_distribution(
    test_dataset_asli,
    title="Image Distribution per Class",
    sort_by_count=False,
    show_percentage=True
)

# Plot sorted distributions
print("\nTraining Set Distribution (Sorted):")
plot_class_distribution(
    train_dataset_asli,
    title="Image Distribution per Class",
    sort_by_count=True,
    show_percentage=True
)

print("\nValidation Set Distribution (Sorted):")
plot_class_distribution(
    val_dataset_asli,
    title="Image Distribution per Class",
    sort_by_count=True,
    show_percentage=True
)

print("\nTest Set Distribution (Sorted):")
plot_class_distribution(
    test_dataset_asli,
    title="Image Distribution per Class",
    sort_by_count=True,
    show_percentage=True
)

print('---------------------------Data Asli + GAN---------------------------')
print("Training Set Distribution:")
plot_class_distribution(
    train_dataset_asli_gan,
    title="Image Distribution per Class (Data Asli + GAN)",
    sort_by_count=False,
    show_percentage=True
)

print("\nValidation Set Distribution:")
plot_class_distribution(
    val_dataset_asli_gan,
    title="Image Distribution per Class (Data Asli + GAN)",
    sort_by_count=False,
    show_percentage=True
)

print("\nTest Set Distribution:")
plot_class_distribution(
    test_dataset_asli_gan,
    title="Image Distribution per Class (Data Asli + GAN)",
    sort_by_count=False,
    show_percentage=True
)

# Plot sorted distributions
print("\nTraining Set Distribution (Sorted):")
plot_class_distribution(
    train_dataset_asli_gan,
    title="Image Distribution per Class (Data Asli + GAN)",
    sort_by_count=True,
    show_percentage=True
)

print("\nValidation Set Distribution (Sorted):")
plot_class_distribution(
    val_dataset_asli_gan,
    title="Image Distribution per Class (Data Asli + GAN)",
    sort_by_count=True,
    show_percentage=True
)

print("\nTest Set Distribution (Sorted):")
plot_class_distribution(
    test_dataset_asli_gan,
    title="Image Distribution per Class (Data Asli + GAN)",
    sort_by_count=True,
    show_percentage=True
)

print('---------------------------Data Asli + GAN + Ratio---------------------------')
print("Training Set Distribution:")
plot_class_distribution(
    train_dataset_asli_gan_ratio,
    title="Image Distribution per Class (Data Asli + GAN + Ratio)",
    sort_by_count=False,
    show_percentage=True
)

print("\nValidation Set Distribution:")
plot_class_distribution(
    val_dataset_asli_gan_ratio,
    title="Image Distribution per Class (Data Asli + GAN + Ratio)",
    sort_by_count=False,
    show_percentage=True
)

print("\nTest Set Distribution:")
plot_class_distribution(
    test_dataset_asli_gan_ratio,
    title="Image Distribution per Class (Data Asli + GAN + Ratio)",
    sort_by_count=False,
    show_percentage=True
)

# Plot sorted distributions
print("\nTraining Set Distribution (Sorted):")
plot_class_distribution(
    train_dataset_asli_gan_ratio,
    title="Image Distribution per Class (Data Asli + GAN + Ratio)",
    sort_by_count=True,
    show_percentage=True
)

print("\nValidation Set Distribution (Sorted):")
plot_class_distribution(
    val_dataset_asli_gan_ratio,
    title="Image Distribution per Class (Data Asli + GAN + Ratio)",
    sort_by_count=True,
    show_percentage=True
)

print("\nTest Set Distribution (Sorted):")
plot_class_distribution(
    test_dataset_asli_gan_ratio,
    title="Image Distribution per Class (Data Asli + GAN + Ratio)",
    sort_by_count=True,
    show_percentage=True
)

In [ ]:
def plot_samples_by_split(dataset, n_samples=5, title=None):
    """
    Plot sample images from each class in a dataset split.

    Args:
        dataset (WasteDataset): The dataset to sample from
        n_samples (int): Number of samples to show per class
        title (str): Optional title for the plot
    """
    sampled_images = defaultdict(list)

    # Sample images for each class
    for label in range(len(dataset.CLASS_NAMES)):
        # Get indices for current class
        class_indices = [i for i, l in enumerate(dataset.labels) if l == label]
        # Randomly sample n_samples images
        if len(class_indices) > 0:
            sampled_indices = random.sample(class_indices, min(n_samples, len(class_indices)))
            sampled_images[label] = sampled_indices

    # Create the plot
    nrows = len(dataset.CLASS_NAMES)
    fig, axs = plt.subplots(nrows, n_samples + 1, figsize=(n_samples * 2.5, nrows * 2))

    if title is None:
        title = f"Sample Images - {dataset.split.capitalize()} Split"
    fig.suptitle(title, fontsize=16)

    # Plot images for each class
    for i, (label, indices) in enumerate(sampled_images.items()):
        class_name = dataset.CLASS_NAMES[label]
        # Add class name
        axs[i, 0].text(0.5, 0.5, class_name, fontsize=12, ha="center", va="center")
        axs[i, 0].axis('off')

        # Plot sample images
        for j, idx in enumerate(indices):
            ax = axs[i, j + 1]
            try:
                img_path = dataset.data[idx]
                img = Image.open(img_path).convert("RGB")
                ax.imshow(img)
                ax.axis('off')
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
                ax.axis('off')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot samples from each split
print('---------------------------Data Asli---------------------------')
print("Training Set Samples:")
plot_samples_by_split(train_dataset_asli, n_samples=5, title="Training Set Samples (Data Asli)")

print("\nValidation Set Samples:")
plot_samples_by_split(val_dataset_asli, n_samples=5, title="Validation Set Samples (Data Asli)")

print("\nTest Set Samples:")
plot_samples_by_split(test_dataset_asli, n_samples=5, title="Test Set Samples (Data Asli)")
print('---------------------------Data Asli + GAN---------------------------')
plot_samples_by_split(train_dataset_asli_gan, n_samples=5, title="Training Set Samples (Data Asli + GAN)")

print("\nValidation Set Samples:")
plot_samples_by_split(val_dataset_asli_gan, n_samples=5, title="Validation Set Samples (Data Asli + GAN)")

print("\nTest Set Samples:")
plot_samples_by_split(test_dataset_asli_gan, n_samples=5, title="Test Set Samples (Data Asli + GAN)")
print('---------------------------Data Asli + GAN + Ratio---------------------------')
plot_samples_by_split(train_dataset_asli_gan, n_samples=5, title="Training Set Samples (Data Asli + GAN + Ratio)")

print("\nValidation Set Samples:")
plot_samples_by_split(val_dataset_asli_gan, n_samples=5, title="Validation Set Samples (Data Asli + GAN + Ratio)")

print("\nTest Set Samples:")
plot_samples_by_split(test_dataset_asli_gan, n_samples=5, title="Test Set Samples (Data Asli + GAN + Ratio)")

# Image Sampling

# EFFICIENT NET V2

In [ ]:
# import os
# import torch
# import torch.nn as nn
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import classification_report, confusion_matrix
# import timm
# from tqdm.auto import tqdm
# from torch.optim.lr_scheduler import ReduceLROnPlateau

# # Device Configuration
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Class names for L1, L2, L3
# class_names = ['L1', 'L2', 'L3']
# num_classes = len(class_names)

# def compute_class_weights(labels, num_classes):
#     class_counts = np.bincount(labels, minlength=num_classes)
#     weights = 1.0 / class_counts
#     weights = weights / weights.sum() * num_classes
#     return torch.tensor(weights, dtype=torch.float).to(device)

# def plot_metrics(history):
#     epochs = range(1, len(history['train_loss']) + 1)
#     plt.figure(figsize=(12, 5))

#     plt.subplot(1, 2, 1)
#     plt.plot(epochs, history['train_loss'], label="Train")
#     plt.plot(epochs, history['val_loss'], label="Validation")
#     plt.title("Loss")
#     plt.xlabel("Epoch")
#     plt.ylabel("Loss")
#     plt.legend()

#     plt.subplot(1, 2, 2)
#     plt.plot(epochs, history['train_acc'], label="Train")
#     plt.plot(epochs, history['val_acc'], label="Validation")
#     plt.title("Accuracy")
#     plt.xlabel("Epoch")
#     plt.ylabel("Accuracy (%)")
#     plt.legend()

#     plt.tight_layout()
#     plt.show()


In [ ]:
# class EarlyStopping:
#     def __init__(self, patience=10, verbose=False):
#         self.patience = patience
#         self.verbose = verbose
#         self.counter = 0
#         self.best_loss = None
#         self.early_stop = False

#     def __call__(self, val_loss):
#         if self.best_loss is None:
#             self.best_loss = val_loss
#         elif val_loss >= self.best_loss:
#             self.counter += 1
#             if self.verbose:
#                 print(f"EarlyStopping counter: {self.counter}/{self.patience}")
#             if self.counter >= self.patience:
#                 self.early_stop = True
#         else:
#             self.best_loss = val_loss
#             self.counter = 0

# # early_stopping = EarlyStopping(patience=EARLY_STOPPING_PATIENCE, verbose=True)


In [ ]:
# def evaluate_model(model, test_loader):
#     model.eval()
#     all_preds = []
#     all_labels = []

#     with torch.no_grad():
#         for images, labels in tqdm(test_loader, desc="Evaluating"):
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images)
#             _, predicted = torch.max(outputs, 1)
#             all_preds.extend(predicted.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())

#     return np.array(all_preds), np.array(all_labels)

# def plot_confusion_matrix(y_true, y_pred, class_names):
#     # Plot normalized confusion matrix
#     cm_norm = confusion_matrix(y_true, y_pred, normalize='true') * 100
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm_norm, annot=True, fmt='.1f', cmap='Blues',
#                 xticklabels=class_names, yticklabels=class_names)
#     plt.title("Normalized Confusion Matrix (%)")
#     plt.xlabel("Predicted")
#     plt.ylabel("True")
#     plt.tight_layout()
#     plt.show()

#     # Plot confusion matrix with actual counts
#     cm = confusion_matrix(y_true, y_pred)
#     plt.figure(figsize=(10, 8))
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
#                 xticklabels=class_names, yticklabels=class_names)

#     # Add row and column totals
#     row_sums = cm.sum(axis=1)
#     col_sums = cm.sum(axis=0)

#     # Add row totals
#     for i in range(len(class_names)):
#         plt.text(len(class_names) + 0.5, i + 0.5, f'Total: {row_sums[i]}',
#                 ha='left', va='center')

#     # Add column totals
#     for i in range(len(class_names)):
#         plt.text(i + 0.5, len(class_names) + 0.5, f'Total: {col_sums[i]}',
#                 ha='center', va='bottom')

#     plt.title("Confusion Matrix (Counts)")
#     plt.xlabel("Predicted")
#     plt.ylabel("True")
#     plt.tight_layout()
#     plt.show()

# def plot_classification_metrics(y_true, y_pred, class_names):
#     report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
#     metrics = ['precision', 'recall', 'f1-score']

#     # Prepare data for plotting
#     class_metrics = {metric: [] for metric in metrics}
#     for class_name in class_names:
#         for metric in metrics:
#             class_metrics[metric].append(report[class_name][metric])

#     # Plot metrics
#     fig, axes = plt.subplots(1, 3, figsize=(15, 5))
#     for ax, metric in zip(axes, metrics):
#         sns.barplot(x=class_metrics[metric], y=class_names, ax=ax, palette="viridis")
#         ax.set_title(f"{metric.capitalize()}")
#         ax.set_xlim(0, 1)
#         for i, v in enumerate(class_metrics[metric]):
#             ax.text(v + 0.01, i, f"{v:.2%}", va="center")

#     plt.tight_layout()
#     plt.show()

#     # Print detailed report
#     print("\nClassification Report:")
#     print(classification_report(y_true, y_pred, target_names=class_names))

#     # Print class distribution
#     print("\nClass Distribution and Metrics:")
#     for i, class_name in enumerate(class_names):
#         true_count = sum(y_true == i)
#         pred_count = sum(y_pred == i)
#         tp = sum((y_true == i) & (y_pred == i))
#         recall = tp / true_count * 100 if true_count > 0 else 0
#         precision = tp / pred_count * 100 if pred_count > 0 else 0
#         print(f"{class_name}:")
#         print(f"  True samples: {true_count}")
#         print(f"  Predicted samples: {pred_count}")
#         print(f"  True Positives: {tp}")
#         print(f"  Recall: {recall:.2f}%")
#         print(f"  Precision: {precision:.2f}%")
#         print()

In [ ]:
# def train_model(
#     model,
#     train_loader,
#     val_loader,
#     early_stopping,
#     lr=0.001,
#     weight_decay=1e-4,
#     best_model_path="best_model.pth",
#     max_epochs=100,  # Batas maksimum epoch
#     val_loss_threshold=0.5  # Stop jika val loss < 0.5
# ):
#     model.to(device)
#     history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

#     # Compute class weights
#     class_weights = compute_class_weights(train_loader.dataset.labels, num_classes)
#     criterion = nn.CrossEntropyLoss(weight=class_weights)
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
#     scheduler = ReduceLROnPlateau(optimizer, patience=3, factor=0.5, verbose=True)

#     best_val_acc = 0
#     epoch = 0

#     best_val_loss = float("inf")  # Tambahkan ini di awal sebelum while loop

#     while epoch < max_epochs:
#         epoch += 1

#         # Training phase
#         model.train()
#         train_loss, train_correct, total = 0, 0, 0
#         for images, labels in tqdm(train_loader, desc=f"Epoch {epoch} [Train]"):
#             images, labels = images.to(device), labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#             train_loss += loss.item()
#             preds = outputs.argmax(1)
#             train_correct += (preds == labels).sum().item()
#             total += labels.size(0)

#         # Validation phase
#         model.eval()
#         val_loss, val_correct = 0, 0
#         with torch.no_grad():
#             for images, labels in tqdm(val_loader, desc=f"Epoch {epoch} [Val]"):
#                 images, labels = images.to(device), labels.to(device)
#                 outputs = model(images)
#                 loss = criterion(outputs, labels)

#                 val_loss += loss.item()
#                 preds = outputs.argmax(1)
#                 val_correct += (preds == labels).sum().item()

#         # Calculate metrics
#         epoch_train_loss = train_loss / len(train_loader)
#         epoch_val_loss = val_loss / len(val_loader)
#         epoch_train_acc = 100 * train_correct / total
#         epoch_val_acc = 100 * val_correct / len(val_loader.dataset)

#         # Update learning rate
#         scheduler.step(epoch_val_loss)

#         # Save metrics
#         history['train_loss'].append(epoch_train_loss)
#         history['val_loss'].append(epoch_val_loss)
#         history['train_acc'].append(epoch_train_acc)
#         history['val_acc'].append(epoch_val_acc)

#         print(f"Epoch {epoch}")
#         print(f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%")
#         print(f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%")

#         # Save best model
#         # Based on Accuracy
#         # if epoch_val_acc > best_val_acc:
#         #     best_val_acc = epoch_val_acc
#         #     torch.save(model.state_dict(), best_model_path)
#         #     print(f"New best model saved! (Val Acc: {best_val_acc:.2f}%)")

#         # Based on Loss
#         if epoch_val_loss < best_val_loss:
#             best_val_loss = epoch_val_loss
#             torch.save(model.state_dict(), best_model_path)
#             print(f"New best model saved! (Val Loss: {best_val_loss:.4f})")

#         # # Stop if val loss is below threshold
#         # if epoch_val_loss < val_loss_threshold:
#         #     print(f"Validation loss < {val_loss_threshold}. Stopping training.")
#         #     break

#         # Early stopping check
#         early_stopping(epoch_val_loss)
#         if early_stopping.early_stop:
#             print("Early stopping triggered.")
#             break

#     return model, history


#### Data Asli

In [ ]:
# print('---------------------------Data Asli---------------------------')
# # Initialize a NEW model for Data Asli
# model_asli = timm.create_model('resnet50', pretrained=False)
# model_asli.fc = nn.Linear(model_asli.fc.in_features, num_classes)

# early_stopping_asli = EarlyStopping(patience=15, verbose=True)
# model_asli, history_asli = train_model(
#     model=model_asli, # Use the new model_asli
#     train_loader=train_loader_asli,
#     val_loader=val_loader_asli,
#     early_stopping=early_stopping_asli,
#     weight_decay=1e-4,
#     best_model_path="best_model_asli.pth"
# )

# # Plot training history for Data Asli
# plot_metrics(history_asli)

# # Evaluate on test set for Data Asli
# y_pred_asli, y_true_asli = evaluate_model(model_asli, test_loader_asli)

# # Plot confusion matrix and classification metrics for Data Asli
# plot_confusion_matrix(y_true_asli, y_pred_asli, class_names)
# plot_classification_metrics(y_true_asli, y_pred_asli, class_names)


#### Model Data Asli + GAN

In [ ]:
# print('---------------------------Data Asli + GAN---------------------------')
# # Initialize a NEW model for Data Asli + GAN
# model_asli_gan = timm.create_model('resnet50', pretrained=False)
# model_asli_gan.fc = nn.Linear(model_asli_gan.fc.in_features, num_classes)

# early_stopping_asli_gan = EarlyStopping(patience=15, verbose=True)
# model_asli_gan, history_asli_gan = train_model(
#     model=model_asli_gan, # Use the new model_asli_gan
#     train_loader=train_loader_asli_gan,
#     val_loader=val_loader_asli_gan,
#     early_stopping=early_stopping_asli_gan,
#     weight_decay=1e-4,
#     best_model_path="best_model_asli_gan.pth"
# )

# # Plot training history for Data Asli + GAN
# plot_metrics(history_asli_gan)

# # Evaluate on test set for Data Asli + GAN
# y_pred_asli_gan, y_true_asli_gan = evaluate_model(model_asli_gan, test_loader_asli_gan)

# # Plot confusion matrix and classification metrics for Data Asli + GAN
# plot_confusion_matrix(y_true_asli_gan, y_pred_asli_gan, class_names)
# plot_classification_metrics(y_true_asli_gan, y_pred_asli_gan, class_names)

#### Model Data Asli Gan Ratio

In [ ]:
# print('---------------------------Data Asli + GAN + Ratio---------------------------')
# # Initialize a NEW model for Data Asli + GAN + Ratio
# model_asli_gan_ratio = timm.create_model('resnet50', pretrained=False)
# model_asli_gan_ratio.fc = nn.Linear(model_asli_gan_ratio.fc.in_features, num_classes)

# early_stopping_asli_gan_ratio = EarlyStopping(patience=15, verbose=True)

# model_asli_gan_ratio, history_asli_gan_ratio = train_model(
#     model=model_asli_gan_ratio, # Use the new model_asli_gan_ratio
#     train_loader=train_loader_asli_gan_ratio,
#     val_loader=val_loader_asli_gan_ratio,
#     early_stopping=early_stopping_asli_gan_ratio,
#     weight_decay=1e-4,
#     best_model_path="best_model_asli_gan_ratio.pth"
# )

# # Plot training history for Data Asli + GAN + Ratio
# plot_metrics(history_asli_gan_ratio)

# # Evaluate on test set for Data Asli + GAN + Ratio
# y_pred_asli_gan_ratio, y_true_asli_gan_ratio = evaluate_model(model_asli_gan_ratio, test_loader_asli_gan_ratio)

# # Plot confusion matrix and classification metrics for Data Asli + GAN + Ratio
# plot_confusion_matrix(y_true_asli_gan_ratio, y_pred_asli_gan_ratio, class_names)
# plot_classification_metrics(y_true_asli_gan_ratio, y_pred_asli_gan_ratio, class_names)

# CNN

In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import timm
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Class names for L1, L2, L3
class_names = ['L1', 'L2', 'L3']
num_classes = len(class_names)

def compute_class_weights(labels, num_classes):
    class_counts = np.bincount(labels, minlength=num_classes)
    weights = 1.0 / class_counts
    weights = weights / weights.sum() * num_classes
    return torch.tensor(weights, dtype=torch.float).to(device)

def plot_metrics(history):
    epochs = range(1, len(history['train_loss']) + 1)
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], label="Train")
    plt.plot(epochs, history['val_loss'], label="Validation")
    plt.title("Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_acc'], label="Train")
    plt.plot(epochs, history['val_acc'], label="Validation")
    plt.title("Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy (%)")
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
# # OG Early Stopping
# class EarlyStopping:
#     def __init__(self, patience=10, verbose=False):
#         self.patience = patience
#         self.verbose = verbose
#         self.counter = 0
#         self.best_loss = None
#         self.early_stop = False

#     def __call__(self, val_loss):
#         if self.best_loss is None:
#             self.best_loss = val_loss
#         elif val_loss >= self.best_loss:
#             self.counter += 1
#             if self.verbose:
#                 print(f"EarlyStopping counter: {self.counter}/{self.patience}")
#             if self.counter >= self.patience:
#                 self.early_stop = True
#         else:
#             self.best_loss = val_loss
#             self.counter = 0

# # early_stopping = EarlyStopping(patience=EARLY_STOPPING_PATIENCE, verbose=True)


In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, min_delta=1e-3): # Tambahkan min_delta
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.min_delta = min_delta # Inisialisasi min_delta

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        # Perubahan di sini: val_loss harus kurang dari (self.best_loss - self.min_delta) untuk dianggap sebagai peningkatan
        elif val_loss >= self.best_loss - self.min_delta: # Jika tidak ada penurunan yang signifikan
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else: # Ada penurunan yang signifikan
            self.best_loss = val_loss
            self.counter = 0

In [ ]:
# --- Fungsi evaluate_model yang dimodifikasi ---
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    mispredicted_data = [] # List untuk menyimpan data gambar yang salah diprediksi

    # Dapatkan akses ke dataset untuk mengambil jalur gambar
    # Asumsi test_loader.dataset adalah instance dari kelas Dataset kustom Anda
    test_dataset = test_loader.dataset

    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(test_loader, desc="Evaluating")):
            images_on_device, labels_on_device = images.to(device), labels.to(device)
            outputs = model(images_on_device)
            _, predicted = torch.max(outputs, 1)

            # Iterasi melalui setiap sampel dalam batch
            for j in range(len(labels_on_device)):
                true_label = labels_on_device[j].item()
                pred_label = predicted[j].item()

                all_preds.append(pred_label)
                all_labels.append(true_label)

                # Jika prediksi salah, simpan informasi gambar
                if true_label != pred_label:
                    # Ambil indeks global sampel di dataset
                    # Ini sedikit tricky karena test_loader mengembalikan batch
                    # Anda perlu cara untuk memetakan kembali ke indeks asli dataset
                    # Cara paling mudah adalah jika Dataset Anda menyimpan path
                    # atau jika Anda bisa menghitung indeks global dari batch_idx dan item_idx
                    # Untuk kesederhanaan, kita akan mengasumsikan test_dataset.data menyimpan jalur gambar
                    # dan menggunakan i * batch_size + j sebagai indeks.
                    # PERHATIAN: Ini mungkin tidak 100% akurat jika test_loader tidak mengembalikan item secara berurutan
                    # atau jika ada filter di dataset. Pastikan test_loader shuffle=False.

                    # Cara yang lebih aman adalah dengan mengembalikan tensor gambar itu sendiri
                    # dari evaluate_model, atau menyimpan indeks asli di __getitem__ Dataset.
                    # Untuk saat ini, mari kita asumsikan test_dataset.data bisa diakses seperti ini:
                    original_image_path = test_dataset.data[i * test_loader.batch_size + j]

                    mispredicted_data.append({
                        'image_path': original_image_path,
                        'true_label': true_label,
                        'predicted_label': pred_label
                    })

    return np.array(all_preds), np.array(all_labels), mispredicted_data

# --- Fungsi baru untuk plotting gambar yang salah diprediksi ---
def plot_mispredicted_images(mispredicted_data, class_names, num_images_to_show=10):
    if not mispredicted_data:
        print("No mispredicted images to show.")
        return

    print(f"\n--- Showing {min(num_images_to_show, len(mispredicted_data))} Mispredicted Images ---")

    # Ambil beberapa sampel acak dari gambar yang salah diprediksi
    display_data = random.sample(mispredicted_data, min(num_images_to_show, len(mispredicted_data)))

    fig, axes = plt.subplots(1, len(display_data), figsize=(len(display_data) * 3, 4))
    if len(display_data) == 1:
        axes = [axes] # Pastikan axes selalu iterable

    for i, data in enumerate(display_data):
        img_path = data['image_path']
        true_label = data['true_label']
        predicted_label = data['predicted_label']

        try:
            img = Image.open(img_path).convert('RGB')
            axes[i].imshow(img)
            axes[i].set_title(
                f"True: {class_names[true_label]}\nPred: {class_names[predicted_label]}",
                color='red' # Untuk menandakan kesalahan
            )
            axes[i].axis('off')
        except Exception as e:
            print(f"Error loading image {img_path} for plotting: {e}")
            axes[i].set_title("Error loading image")
            axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# --- Fungsi plot_confusion_matrix dan plot_classification_metrics tetap sama ---
def plot_confusion_matrix(y_true, y_pred, class_names): #
    # Plot normalized confusion matrix
    cm_norm = confusion_matrix(y_true, y_pred, normalize='true') * 100
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_norm, annot=True, fmt='.1f', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title("Normalized Confusion Matrix (%)")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.show()

    # Plot confusion matrix with actual counts
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)

    # Add row and column totals
    row_sums = cm.sum(axis=1)
    col_sums = cm.sum(axis=0)

    # Add row totals
    for i in range(len(class_names)):
        plt.text(len(class_names) + 0.5, i + 0.5, f'Total: {row_sums[i]}',
                ha='left', va='center')

    # Add column totals
    for i in range(len(class_names)):
        plt.text(i + 0.5, len(class_names) + 0.5, f'Total: {col_sums[i]}',
                ha='center', va='bottom')

    plt.title("Confusion Matrix (Counts)")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.show()

def plot_classification_metrics(y_true, y_pred, class_names): #
    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    metrics = ['precision', 'recall', 'f1-score']

    # Prepare data for plotting
    class_metrics = {metric: [] for metric in metrics}
    for class_name in class_names:
        for metric in metrics:
            class_metrics[metric].append(report[class_name][metric])

    # Plot metrics
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    for ax, metric in zip(axes, metrics):
        sns.barplot(x=class_metrics[metric], y=class_names, ax=ax, palette="viridis")
        ax.set_title(f"{metric.capitalize()}")
        ax.set_xlim(0, 1)
        for i, v in enumerate(class_metrics[metric]):
            ax.text(v + 0.01, i, f"{v:.2%}", va="center")

    plt.tight_layout()
    plt.show()

    # Print detailed report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # Print class distribution
    print("\nClass Distribution and Metrics:")
    for i, class_name in enumerate(class_names):
        true_count = sum(y_true == i)
        pred_count = sum(y_pred == i)
        tp = sum((y_true == i) & (y_pred == i))
        recall = tp / true_count * 100 if true_count > 0 else 0
        precision = tp / pred_count * 100 if pred_count > 0 else 0
        print(f"{class_name}:")
        print(f"  True samples: {true_count}")
        print(f"  Predicted samples: {pred_count}")
        print(f"  True Positives: {tp}")
        print(f"  Recall: {recall:.2f}%")
        print(f"  Precision: {precision:.2f}%")
        print()

In [ ]:
# ### ORIGINAL
# def train_model(
#     model,
#     train_loader,
#     val_loader,
#     early_stopping,
#     lr=0.001,
#     weight_decay=1e-4,
#     best_model_path="best_model.pth",
#     max_epochs=100,  # Batas maksimum epoch
#     val_loss_threshold=0.5  # Stop jika val loss < 0.5
# ):
#     model.to(device)
#     history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

#     # Compute class weights
#     class_weights = compute_class_weights(train_loader.dataset.labels, num_classes)
#     criterion = nn.CrossEntropyLoss(weight=class_weights)
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
#     scheduler = ReduceLROnPlateau(optimizer, patience=3, factor=0.5, verbose=True)

#     best_val_acc = 0
#     epoch = 0

#     best_val_loss = float("inf")  # Tambahkan ini di awal sebelum while loop

#     while epoch < max_epochs:
#         epoch += 1

#         # Training phase
#         model.train()
#         train_loss, train_correct, total = 0, 0, 0
#         for images, labels in tqdm(train_loader, desc=f"Epoch {epoch} [Train]"):
#             images, labels = images.to(device), labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#             train_loss += loss.item()
#             preds = outputs.argmax(1)
#             train_correct += (preds == labels).sum().item()
#             total += labels.size(0)

#         # Validation phase
#         model.eval()
#         val_loss, val_correct = 0, 0
#         with torch.no_grad():
#             for images, labels in tqdm(val_loader, desc=f"Epoch {epoch} [Val]"):
#                 images, labels = images.to(device), labels.to(device)
#                 outputs = model(images)
#                 loss = criterion(outputs, labels)

#                 val_loss += loss.item()
#                 preds = outputs.argmax(1)
#                 val_correct += (preds == labels).sum().item()

#         # Calculate metrics
#         epoch_train_loss = train_loss / len(train_loader)
#         epoch_val_loss = val_loss / len(val_loader)
#         epoch_train_acc = 100 * train_correct / total
#         epoch_val_acc = 100 * val_correct / len(val_loader.dataset)

#         # Update learning rate
#         scheduler.step(epoch_val_loss)

#         # Save metrics
#         history['train_loss'].append(epoch_train_loss)
#         history['val_loss'].append(epoch_val_loss)
#         history['train_acc'].append(epoch_train_acc)
#         history['val_acc'].append(epoch_val_acc)

#         print(f"Epoch {epoch}")
#         print(f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%")
#         print(f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%")

#         # Save best model
#         # Based on Accuracy
#         # if epoch_val_acc > best_val_acc:
#         #     best_val_acc = epoch_val_acc
#         #     torch.save(model.state_dict(), best_model_path)
#         #     print(f"New best model saved! (Val Acc: {best_val_acc:.2f}%)")

#         # Based on Loss
#         if epoch_val_loss < best_val_loss:
#             best_val_loss = epoch_val_loss
#             torch.save(model.state_dict(), best_model_path)
#             print(f"New best model saved! (Val Loss: {best_val_loss:.4f})")

#         # # Stop if val loss is below threshold
#         # if epoch_val_loss < val_loss_threshold:
#         #     print(f"Validation loss < {val_loss_threshold}. Stopping training.")
#         #     break

#         # Early stopping check
#         early_stopping(epoch_val_loss)
#         if early_stopping.early_stop:
#             print("Early stopping triggered.")
#             break

#     return model, history


In [ ]:
def train_model(
    model,
    train_loader,
    val_loader,
    early_stopping=None,
    lr=0.001,
    weight_decay=1e-4,
    best_model_path="best_model.pth",
    max_epochs=100,
    val_loss_threshold=0.5
):
    model.to(device)
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

    class_weights = compute_class_weights(train_loader.dataset.labels, num_classes)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, patience=3, factor=0.5, verbose=True)

    # Inisialisasi best_val_loss hanya jika early_stopping diaktifkan
    best_val_loss = float("inf")

    # Tambahkan variabel untuk menyimpan state_dict model terakhir jika early_stopping=None
    last_model_state = None

    epoch = 0

    while epoch < max_epochs:
        epoch += 1

        # Training phase
        model.train()
        train_loss, train_correct, total = 0, 0, 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch} [Train]"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            preds = outputs.argmax(1)
            train_correct += (preds == labels).sum().item()
            total += labels.size(0)

        # Validation phase
        model.eval()
        val_loss, val_correct = 0, 0
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch} [Val]"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = outputs.argmax(1)
                val_correct += (preds == labels).sum().item()

        # Calculate metrics
        epoch_train_loss = train_loss / len(train_loader)
        epoch_val_loss = val_loss / len(val_loader)
        epoch_train_acc = 100 * train_correct / total
        epoch_val_acc = 100 * val_correct / len(val_loader.dataset)

        scheduler.step(epoch_val_loss)

        history['train_loss'].append(epoch_train_loss)
        history['val_loss'].append(epoch_val_loss)
        history['train_acc'].append(epoch_train_acc)
        history['val_acc'].append(epoch_val_acc)

        print(f"Epoch {epoch}")
        print(f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%\n"
              f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%")

        # --- Logika Penyimpanan Model yang Dimodifikasi ---
        if early_stopping is not None:
            # Jika early_stopping aktif, save model terbaik berdasarkan validasi loss
            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                torch.save(model.state_dict(), best_model_path)
                print(f"New best model saved! (Val Loss: {best_val_loss:.4f})")

            # Cek early stopping
            early_stopping(epoch_val_loss)
            if early_stopping.early_stop:
                print("Early stopping triggered.")
                break
        else:
            # Jika early_stopping dinonaktifkan (None), update state_dict model terakhir
            last_model_state = model.state_dict()
            # Di sini kita tidak save ke disk di setiap epoch, hanya di akhir loop
            # Proses save ke disk akan dilakukan di luar loop ini

    # --- Simpan model terakhir ke disk jika early_stopping dinonaktifkan ---
    if early_stopping is None and last_model_state is not None:
        torch.save(last_model_state, best_model_path)
        print(f"Model from last epoch ({max_epochs}) saved to {best_model_path}")
        # Penting: Setelah menyimpan state_dict terakhir, pastikan model dimuat kembali dengan state_dict ini
        # untuk memastikan model yang dikembalikan adalah model terakhir, bukan yang terbaik dari epoch sebelumnya
        model.load_state_dict(last_model_state)
    elif early_stopping is not None: # Jika early_stopping aktif dan model terbaik sudah disimpan
        # Muat kembali model terbaik yang sudah disimpan jika early stopping aktif dan sudah ada model disimpan
        if os.path.exists(best_model_path):
            model.load_state_dict(torch.load(best_model_path))
            print(f"Loaded best model from {best_model_path}")
        else:
            print(f"Warning: Best model not found at {best_model_path}. Returning current model state.")


    return model, history

In [ ]:
# class SimpleCNN(nn.Module):
#     def __init__(self, num_classes=3):
#         super(SimpleCNN, self).__init__()
#         # Layer konvolusi pertama
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
#         self.bn1 = nn.BatchNorm2d(16)
#         # self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # Mengurangi ukuran gambar / 2
#         self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2) # Mengurangi ukuran gambar / 2

#         # Layer konvolusi kedua
#         self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
#         self.bn2 = nn.BatchNorm2d(32)
#         # self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # Mengurangi ukuran gambar / 2
#         self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2) # Mengurangi ukuran gambar / 2

#         # Layer konvolusi ketiga
#         self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
#         self.bn3 = nn.BatchNorm2d(64)
#         # self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) # Mengurangi ukuran gambar / 2
#         self.pool3 = nn.AvgPool2d(kernel_size=2, stride=2) # Mengurangi ukuran gambar / 2

#         # --- PENTING: Hitung ukuran input untuk layer FC berdasarkan IMAGE_SIZE asli Anda ---
#         # Untuk input 256x256 piksel, setelah 3 kali MaxPool2d dengan kernel_size=2, stride=2:
#         # 256 / 2 = 128
#         # 128 / 2 = 64
#         # 64 / 2 = 32
#         # Jadi, output dari conv layer terakhir adalah 64 channel dengan ukuran 32x32
#         self.fc_input_features = 64 * 32 * 32 # 64 channels * 32 * 32 = 65536
#         self.fc = nn.Linear(self.fc_input_features, num_classes)
#         self.dropout = nn.Dropout(0.5) # Tambahkan Dropout dengan probabilitas 0.5

#     def forward(self, x):
#         x = self.pool1(F.relu(self.bn1(self.conv1(x))))
#         x = self.pool2(F.relu(self.bn2(self.conv2(x))))
#         x = self.pool3(F.relu(self.bn3(self.conv3(x))))
#         # x = self.pool1(F.relu(self.conv1(x)))
#         # x = self.pool2(F.relu(self.conv2(x)))
#         # x = self.pool3(F.relu(self.conv3(x)))

#         # Flatten the tensor for the fully connected layer
#         x = x.view(-1, self.fc_input_features)
#         x = self.dropout(x) # Terapkan Dropout sebelum layer FC
#         x = self.fc(x)
#         return x

In [ ]:
# import torch.nn as nn
# import torch.nn.functional as F
# import torch # Pastikan torch diimpor untuk torch.cat

# class SimpleCNN(nn.Module):
#     def __init__(self, num_classes=3):
#         super(SimpleCNN, self).__init__()

#         # Conv Block 1
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
#         self.bn1 = nn.BatchNorm2d(16)
#         self.avgpool1 = nn.AvgPool2d(kernel_size=2, stride=2)
#         self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

#         # Conv Block 2 (in_channels sekarang 16 * 2 = 32)
#         self.conv2 = nn.Conv2d(in_channels=16 * 2, out_channels=32, kernel_size=3, padding=1)
#         self.bn2 = nn.BatchNorm2d(32)
#         self.avgpool2 = nn.AvgPool2d(kernel_size=2, stride=2)
#         self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

#         # Conv Block 3 (in_channels sekarang 32 * 2 = 64)
#         self.conv3 = nn.Conv2d(in_channels=32 * 2, out_channels=64, kernel_size=3, padding=1)
#         self.bn3 = nn.BatchNorm2d(64)
#         self.avgpool3 = nn.AvgPool2d(kernel_size=2, stride=2)
#         self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

#         # Hitung ukuran input untuk layer FC
#         # Output channels dari conv3 adalah 64. Setelah pooling ganda dan concatenate, jadi 64 * 2 = 128 channels.
#         # Ukuran spasial (256x256 -> 128x128 -> 64x64 -> 32x32)
#         self.fc_input_features = (64 * 2) * 32 * 32 # 128 channels * 32 * 32 = 131072
#         self.fc = nn.Linear(self.fc_input_features, num_classes)
#         self.dropout = nn.Dropout(0.5)

#     def forward(self, x):
#         # Blok 1: Conv -> BatchNorm -> ReLU -> (AvgPool, MaxPool) -> Concat
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = F.relu(x)
#         x_avg = self.avgpool1(x)
#         x_max = self.maxpool1(x)
#         x = torch.cat([x_avg, x_max], dim=1) # Concatenate output AvgPool dan MaxPool

#         # Blok 2: Conv (input dari concat sebelumnya) -> BatchNorm -> ReLU -> (AvgPool, MaxPool) -> Concat
#         x = self.conv2(x)
#         x = self.bn2(x)
#         x = F.relu(x)
#         x_avg = self.avgpool2(x)
#         x_max = self.maxpool2(x)
#         x = torch.cat([x_avg, x_max], dim=1) # Concatenate output AvgPool dan MaxPool

#         # Blok 3: Conv (input dari concat sebelumnya) -> BatchNorm -> ReLU -> (AvgPool, MaxPool) -> Concat
#         x = self.conv3(x)
#         x = self.bn3(x)
#         x = F.relu(x)
#         x_avg = self.avgpool3(x)
#         x_max = self.maxpool3(x)
#         x = torch.cat([x_avg, x_max], dim=1) # Concatenate output AvgPool dan MaxPool

#         # Flatten the tensor for the fully connected layer
#         x = x.view(-1, self.fc_input_features)
#         x = self.dropout(x) # Terapkan Dropout sebelum layer FC
#         x = self.fc(x)
#         return x

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch # Pastikan torch diimpor untuk torch.cat

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(SimpleCNN, self).__init__()

        # Conv Block 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.avgpool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Conv Block 2 (in_channels sekarang 16 * 2 = 32)
        self.conv2 = nn.Conv2d(in_channels=16 * 2, out_channels=32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.avgpool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Conv Block 3 (in_channels sekarang 32 * 2 = 64)
        self.conv3 = nn.Conv2d(in_channels=32 * 2, out_channels=64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.avgpool3 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # --- TAMBAHAN: Global Average Pooling ---
        # Ini akan mengambil output dari lapisan konvolusi/pooling terakhir
        # dan mereduksinya menjadi 1x1 spasial per channel.
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1)) # Output 1x1 per channel

        # Hitung ukuran input untuk layer FC setelah Global Average Pooling
        # Output channels dari conv3 adalah 64. Setelah pooling ganda dan concatenate, jadi 64 * 2 = 128 channels.
        # Setelah Global Average Pooling (yang mengubah dimensi spasial menjadi 1x1),
        # input ke FC hanya akan menjadi jumlah channels ini.
        self.fc_input_features = (64 * 2) # Hanya jumlah channels, karena spasial sudah 1x1
        self.fc = nn.Linear(self.fc_input_features, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Blok 1: Conv -> BatchNorm -> ReLU -> (AvgPool, MaxPool) -> Concat
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x_avg = self.avgpool1(x)
        x_max = self.maxpool1(x)
        x = torch.cat([x_avg, x_max], dim=1)

        # Blok 2: Conv -> BatchNorm -> ReLU -> (AvgPool, MaxPool) -> Concat
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x_avg = self.avgpool2(x)
        x_max = self.maxpool2(x)
        x = torch.cat([x_avg, x_max], dim=1)

        # Blok 3: Conv -> BatchNorm -> ReLU -> (AvgPool, MaxPool) -> Concat
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x_avg = self.avgpool3(x)
        x_max = self.maxpool3(x)
        x = torch.cat([x_avg, x_max], dim=1)

        # --- APLIKASIKAN GLOBAL AVERAGE POOLING DI SINI ---
        x = self.global_avg_pool(x) # Output: (batch_size, 128, 1, 1)

        # Flatten the tensor for the fully connected layer
        # Sekarang flatten lebih sederhana karena spasial sudah 1x1
        x = x.view(x.size(0), -1) # Akan menjadi (batch_size, 128)

        x = self.dropout(x) # Terapkan Dropout sebelum layer FC
        x = self.fc(x)
        return x

In [ ]:
# # --- Di bagian bawah notebook Anda, setelah fungsi train_model, evaluate_model, dll. ---

# print('---------------------------Data Asli (CNN Biasa)---------------------------')
# # Inisialisasi model SimpleCNN baru untuk Data Asli
# model_asli_cnn = SimpleCNN(num_classes=NUM_CLASSES)

# early_stopping_asli_cnn = EarlyStopping(patience=10, verbose=True)
# model_asli_cnn, history_asli_cnn = train_model(
#     model=model_asli_cnn,
#     train_loader=train_loader_asli,
#     val_loader=val_loader_asli,
#     early_stopping=early_stopping_asli_cnn,
#     weight_decay=1e-4, # Anda mungkin ingin menyesuaikan weight_decay dan lr untuk CNN biasa
#     best_model_path="best_model_asli_cnn.pth"
# )

# plot_metrics(history_asli_cnn)
# y_pred_asli_cnn, y_true_asli_cnn = evaluate_model(model_asli_cnn, test_loader_asli)
# plot_confusion_matrix(y_true_asli_cnn, y_pred_asli_cnn, class_names)
# plot_classification_metrics(y_true_asli_cnn, y_pred_asli_cnn, class_names)


# print('---------------------------Data Asli + GAN (CNN Biasa)---------------------------')
# # Inisialisasi model SimpleCNN baru untuk Data Asli + GAN
# model_asli_gan_cnn = SimpleCNN(num_classes=NUM_CLASSES)

# early_stopping_asli_gan_cnn = EarlyStopping(patience=10, verbose=True)
# model_asli_gan_cnn, history_asli_gan_cnn = train_model(
#     model=model_asli_gan_cnn,
#     train_loader=train_loader_asli_gan,
#     val_loader=val_loader_asli_gan,
#     early_stopping=early_stopping_asli_gan_cnn,
#     weight_decay=1e-4,
#     best_model_path="best_model_asli_gan_cnn.pth"
# )

# plot_metrics(history_asli_gan_cnn)
# y_pred_asli_gan_cnn, y_true_asli_gan_cnn = evaluate_model(model_asli_gan_cnn, test_loader_asli_gan)
# plot_confusion_matrix(y_true_asli_gan_cnn, y_pred_asli_gan_cnn, class_names)
# plot_classification_metrics(y_true_asli_gan_cnn, y_pred_asli_gan_cnn, class_names)


# print('---------------------------Data Asli + GAN + Ratio (CNN Biasa)---------------------------')
# # Inisialisasi model SimpleCNN baru untuk Data Asli + GAN + Ratio
# model_asli_gan_ratio_cnn = SimpleCNN(num_classes=NUM_CLASSES)

# early_stopping_asli_gan_ratio_cnn = EarlyStopping(patience=10, verbose=True)
# model_asli_gan_ratio_cnn, history_asli_gan_ratio_cnn = train_model(
#     model=model_asli_gan_ratio_cnn,
#     train_loader=train_loader_asli_gan_ratio,
#     val_loader=val_loader_asli_gan_ratio,
#     early_stopping=early_stopping_asli_gan_ratio_cnn,
#     weight_decay=1e-4,
#     best_model_path="best_model_asli_gan_ratio_cnn.pth"
# )

# plot_metrics(history_asli_gan_ratio_cnn)
# y_pred_asli_gan_ratio_cnn, y_true_asli_gan_ratio_cnn = evaluate_model(model_asli_gan_ratio_cnn, test_loader_asli_gan_ratio)
# plot_confusion_matrix(y_true_asli_gan_ratio_cnn, y_pred_asli_gan_ratio_cnn, class_names)
# plot_classification_metrics(y_true_asli_gan_ratio_cnn, y_pred_asli_gan_ratio_cnn, class_names)

In [ ]:
def run_cnn_experiment_automated_naming(patience_value, patience_name_suffix,
                                       # DataLoaders sesuai penamaan Anda
                                       train_loader_asli, val_loader_asli, test_loader_asli,
                                       train_loader_asli_gan, val_loader_asli_gan, test_loader_asli_gan,
                                       train_loader_asli_gan_ratio, val_loader_asli_gan_ratio, test_loader_asli_gan_ratio,
                                       class_names, num_classes, device, weight_decay, num_images_to_show=37):
    """
    Menjalankan eksperimen training dan evaluasi untuk CNN Biasa dengan penamaan variabel otomatis
    berdasarkan konfigurasi early stopping, dan mengembalikan hasilnya.
    """

    all_results_for_this_patience = {}

    print(f'\n---------------------------CNN Biasa (Early Stopping {patience_name_suffix})---------------------------')

    # --- Case: Data Asli ---
    print(f'---------------------------Data Asli (CNN Biasa - ES {patience_name_suffix})---------------------------')
    # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
    early_stopping_instance_asli = None
    if patience_value is not None:
        early_stopping_instance_asli = EarlyStopping(patience=patience_value, verbose=True)

    model_asli = SimpleCNN(num_classes=num_classes).to(device)
    best_model_path_asli = f"best_model_asli_cnn_{patience_name_suffix}.pth"

    model_asli, history_asli = train_model(
        model=model_asli,
        train_loader=train_loader_asli,
        val_loader=val_loader_asli,
        early_stopping=early_stopping_instance_asli, # Gunakan instance baru
        weight_decay=weight_decay,
        best_model_path=best_model_path_asli
    )

    plot_metrics(history_asli)
    y_pred_asli, y_true_asli, mispredicted_data_asli = evaluate_model(model_asli, test_loader_asli)
    plot_confusion_matrix(y_true_asli, y_pred_asli, class_names)
    plot_classification_metrics(y_true_asli, y_pred_asli, class_names)
    plot_mispredicted_images(mispredicted_data_asli, class_names, num_images_to_show=num_images_to_show)

    all_results_for_this_patience['asli'] = {
        'model': model_asli, 'history': history_asli, 'y_pred': y_pred_asli,
        'y_true': y_true_asli, 'mispredicted_data': mispredicted_data_asli,
        'best_model_path': best_model_path_asli
    }

    # --- Case: Data Asli + GAN ---
    print(f'\n---------------------------Data Asli + GAN (CNN Biasa - ES {patience_name_suffix})---------------------------')
    # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
    early_stopping_instance_asli_gan = None
    if patience_value is not None:
        early_stopping_instance_asli_gan = EarlyStopping(patience=patience_value, verbose=True)

    model_asli_gan = SimpleCNN(num_classes=num_classes).to(device)
    best_model_path_asli_gan = f"best_model_asli_gan_cnn_{patience_name_suffix}.pth"

    model_asli_gan, history_asli_gan = train_model(
        model=model_asli_gan,
        train_loader=train_loader_asli_gan,
        val_loader=val_loader_asli_gan,
        early_stopping=early_stopping_instance_asli_gan, # Gunakan instance baru
        weight_decay=weight_decay,
        best_model_path=best_model_path_asli_gan
    )

    plot_metrics(history_asli_gan)
    y_pred_asli_gan, y_true_asli_gan, mispredicted_data_asli_gan = evaluate_model(model_asli_gan, test_loader_asli_gan)
    plot_confusion_matrix(y_true_asli_gan, y_pred_asli_gan, class_names)
    plot_classification_metrics(y_true_asli_gan, y_pred_asli_gan, class_names)
    plot_mispredicted_images(mispredicted_data_asli_gan, class_names, num_images_to_show=num_images_to_show)

    all_results_for_this_patience['asli_gan'] = {
        'model': model_asli_gan, 'history': history_asli_gan, 'y_pred': y_pred_asli_gan,
        'y_true': y_true_asli_gan, 'mispredicted_data': mispredicted_data_asli_gan,
        'best_model_path': best_model_path_asli_gan
    }

    # --- Case: Data Asli + GAN + Ratio ---
    print(f'\n---------------------------Data Asli + GAN + Ratio (CNN Biasa - ES {patience_name_suffix})---------------------------')
    # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
    early_stopping_instance_asli_gan_ratio = None
    if patience_value is not None:
        early_stopping_instance_asli_gan_ratio = EarlyStopping(patience=patience_value, verbose=True)

    model_asli_gan_ratio = SimpleCNN(num_classes=num_classes).to(device)
    best_model_path_asli_gan_ratio = f"best_model_asli_gan_ratio_cnn_{patience_name_suffix}.pth"

    model_asli_gan_ratio, history_asli_gan_ratio = train_model(
        model=model_asli_gan_ratio,
        train_loader=train_loader_asli_gan_ratio,
        val_loader=val_loader_asli_gan_ratio,
        early_stopping=early_stopping_instance_asli_gan_ratio, # Gunakan instance baru
        weight_decay=weight_decay,
        best_model_path=best_model_path_asli_gan_ratio
    )

    plot_metrics(history_asli_gan_ratio)
    y_pred_asli_gan_ratio, y_true_asli_gan_ratio, mispredicted_data_asli_gan_ratio = evaluate_model(model_asli_gan_ratio, test_loader_asli_gan_ratio)
    plot_confusion_matrix(y_true_asli_gan_ratio, y_pred_asli_gan_ratio, class_names)
    plot_classification_metrics(y_true_asli_gan_ratio, y_pred_asli_gan_ratio, class_names)
    plot_mispredicted_images(mispredicted_data_asli_gan_ratio, class_names, num_images_to_show=num_images_to_show)

    all_results_for_this_patience['asli_gan_ratio'] = {
        'model': model_asli_gan_ratio, 'history': history_asli_gan_ratio, 'y_pred': y_pred_asli_gan_ratio,
        'y_true': y_true_asli_gan_ratio, 'mispredicted_data': mispredicted_data_asli_gan_ratio,
        'best_model_path': best_model_path_asli_gan_ratio
    }

    return all_results_for_this_patience

# --- Panggilan Fungsi untuk Setiap Konfigurasi Early Stopping (tidak ada perubahan di sini) ---
# ... (kode panggilan fungsi yang sudah ada) ...

# --- Panggilan Fungsi untuk Setiap Konfigurasi Early Stopping ---

# Membuat dictionary global untuk menyimpan semua hasil eksperimen
# Contoh: all_experiment_results['es5']['asli']['model']
all_experiment_results_cnn = {}

# Panggil fungsi untuk setiap early stopping patience yang Anda inginkan
early_stopping_patience_configs = {
    "es5": EARLY_STOPPING_PATIENCE_5,
    "es10": EARLY_STOPPING_PATIENCE_10,
    "es15": EARLY_STOPPING_PATIENCE_15,
    # "es20": EARLY_STOPPING_PATIENCE_20,
    "esNone": EARLY_STOPPING_PATIENCE_NONE
}

for suffix, patience_val in early_stopping_patience_configs.items():
    all_experiment_results_cnn[suffix] = run_cnn_experiment_automated_naming(patience_val, suffix,
                                                                             train_loader_asli, val_loader_asli, test_loader_asli,
                                                                             train_loader_asli_gan, val_loader_asli_gan, test_loader_asli_gan,
                                                                             train_loader_asli_gan_ratio, val_loader_asli_gan_ratio, test_loader_asli_gan_ratio,
                                                                             class_names, NUM_CLASSES, DEVICE, WEIGHT_DECAY)

# Sekarang Anda bisa mengakses hasilnya seperti ini:
# model_es5_asli = all_experiment_results_cnn['es5']['asli']['model']
# history_es10_asli_gan = all_experiment_results_cnn['es10']['asli_gan']['history']
# y_pred_esNone_asli_gan_ratio = all_experiment_results_cnn['esNone']['asli_gan_ratio']['y_pred']


In [ ]:
# import timm # Pastikan timm diimpor
# import torch.nn as nn # Pastikan nn diimpor
# import random # Pastikan random diimpor
# import numpy as np # Pastikan numpy diimpor
# import matplotlib.pyplot as plt # Pastikan matplotlib diimpor
# import seaborn as sns # Pastikan seaborn diimpor

# # Pastikan class_names, NUM_CLASSES, dan device sudah didefinisikan.
# # Pastikan fungsi train_model, evaluate_model, plot_metrics, plot_confusion_matrix,
# # plot_classification_metrics, dan plot_mispredicted_images sudah didefinisikan dengan benar.
# # Pastikan EarlyStopping sudah didefinisikan (dengan min_delta opsional).

# # --- Pastikan DataLoader Anda sudah ada dan terkonfigurasi dengan benar ---
# # train_loader_asli
# # val_loader_asli
# # test_loader_asli
# # train_loader_asli_gan
# # val_loader_asli_gan
# # test_loader_asli_gan
# # train_loader_asli_gan_ratio
# # val_loader_asli_gan_ratio
# # test_loader_asli_gan_ratio


# def run_resnet_experiment_automated_naming(patience_value, patience_name_suffix,
#                                           # DataLoaders sesuai penamaan Anda
#                                           train_loader_asli, val_loader_asli, test_loader_asli,
#                                           train_loader_asli_gan, val_loader_asli_gan, test_loader_asli_gan,
#                                           train_loader_asli_gan_ratio, val_loader_asli_gan_ratio, test_loader_asli_gan_ratio,
#                                           class_names, num_classes, device, weight_decay, num_images_to_show=37):
#     """
#     Menjalankan eksperimen training dan evaluasi untuk ResNet50 dengan penamaan variabel otomatis
#     berdasarkan konfigurasi early stopping.

#     Args:
#         patience_value: Nilai patience untuk EarlyStopping, atau None untuk menonaktifkan.
#         patience_name_suffix: String suffix untuk penamaan variabel (misal: "es5", "esNone").
#         train_loader_asli, val_loader_asli, test_loader_asli: DataLoaders untuk dataset asli.
#         train_loader_asli_gan, val_loader_asli_gan, test_loader_asli_gan: DataLoaders untuk dataset asli + GAN.
#         train_loader_asli_gan_ratio, val_loader_asli_gan_ratio, test_loader_asli_gan_ratio: DataLoaders untuk dataset asli + GAN + Ratio.
#         class_names: List nama kelas.
#         num_classes: Jumlah kelas.
#         device: Device (cuda/cpu) untuk training.
#         weight_decay: Weight decay untuk optimizer.
#         num_images_to_show: Jumlah gambar mispredicted yang akan ditampilkan.
#     """

#     print(f'\n---------------------------ResNet50 (TIMM) (Early Stopping {patience_name_suffix})---------------------------')

#     # --- Case: Data Asli ---
#     print(f'---------------------------Data Asli (ResNet50 - ES {patience_name_suffix})---------------------------')
#     # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
#     early_stopping_instance_asli = None
#     if patience_value is not None:
#         early_stopping_instance_asli = EarlyStopping(patience=patience_value, verbose=True)

#     model_asli_resnet = timm.create_model('resnet50', pretrained=False) # Menggunakan pretrained=False sesuai kode Anda
#     # model_asli_resnet.fc = nn.Linear(model_asli_resnet.fc.in_features, num_classes).to(device) # Mengganti layer .fc untuk ResNet dan pindahkan ke device
#     model_asli_resnet.fc = nn.Sequential(
#         nn.Dropout(0.5), # Probabilitas Dropout, bisa disesuaikan
#         nn.Linear(model_asli_resnet.fc.in_features, num_classes)
#     ).to(device) # Pindahkan seluruh Sequential ke device

#     best_model_path_asli = f"best_model_asli_resnet_{patience_name_suffix}.pth"

#     model_asli_resnet, history_asli_resnet = train_model(
#         model=model_asli_resnet,
#         train_loader=train_loader_asli,
#         val_loader=val_loader_asli,
#         early_stopping=early_stopping_instance_asli, # Gunakan instance baru
#         weight_decay=weight_decay,
#         best_model_path=best_model_path_asli
#     )

#     plot_metrics(history_asli_resnet)
#     y_pred_asli_resnet, y_true_asli_resnet, mispredicted_data_asli_resnet = evaluate_model(model_asli_resnet, test_loader_asli)
#     plot_confusion_matrix(y_true_asli_resnet, y_pred_asli_resnet, class_names)
#     plot_classification_metrics(y_true_asli_resnet, y_pred_asli_resnet, class_names)
#     plot_mispredicted_images(mispredicted_data_asli_resnet, class_names, num_images_to_show=num_images_to_show)

#     # --- Case: Data Asli + GAN ---
#     print(f'\n---------------------------Data Asli + GAN (ResNet50 - ES {patience_name_suffix})---------------------------')
#     # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
#     early_stopping_instance_asli_gan = None
#     if patience_value is not None:
#         early_stopping_instance_asli_gan = EarlyStopping(patience=patience_value, verbose=True)

#     model_asli_gan_resnet = timm.create_model('resnet50', pretrained=False) # Menggunakan pretrained=False sesuai kode Anda
#     # model_asli_gan_resnet.fc = nn.Linear(model_asli_gan_resnet.fc.in_features, num_classes).to(device) # Mengganti layer .fc untuk ResNet dan pindahkan ke device
#     model_asli_gan_resnet.fc = nn.Sequential(
#         nn.Dropout(0.5), # Probabilitas Dropout, bisa disesuaikan
#         nn.Linear(model_asli_gan_resnet.fc.in_features, num_classes)
#     ).to(device)

#     model_asli_gan_resnet, history_asli_gan_resnet = train_model(
#         model=model_asli_gan_resnet,
#         train_loader=train_loader_asli_gan,
#         val_loader=val_loader_asli_gan, # Menggunakan val_loader_asli_gan (sesuai instruksi Anda)
#         early_stopping=early_stopping_instance_asli_gan, # Gunakan instance baru
#         weight_decay=weight_decay,
#         best_model_path=f"best_model_asli_gan_resnet_{patience_name_suffix}.pth"
#     )

#     plot_metrics(history_asli_gan_resnet)
#     y_pred_asli_gan_resnet, y_true_asli_gan_resnet, mispredicted_data_asli_gan_resnet = evaluate_model(model_asli_gan_resnet, test_loader_asli_gan) # Menggunakan test_loader_asli_gan (sesuai instruksi Anda)
#     plot_confusion_matrix(y_true_asli_gan_resnet, y_pred_asli_gan_resnet, class_names)
#     plot_classification_metrics(y_true_asli_gan_resnet, y_pred_asli_gan_resnet, class_names)
#     plot_mispredicted_images(mispredicted_data_asli_gan_resnet, class_names, num_images_to_show=num_images_to_show)

#     # --- Case: Data Asli + GAN + Ratio ---
#     print(f'\n---------------------------Data Asli + GAN + Ratio (ResNet50 - ES {patience_name_suffix})---------------------------')
#     # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
#     early_stopping_instance_asli_gan_ratio = None
#     if patience_value is not None:
#         early_stopping_instance_asli_gan_ratio = EarlyStopping(patience=patience_value, verbose=True)

#     model_asli_gan_ratio_resnet = timm.create_model('resnet50', pretrained=False) # Menggunakan pretrained=False sesuai kode Anda
#     # model_asli_gan_ratio_resnet.fc = nn.Linear(model_asli_gan_ratio_resnet.fc.in_features, num_classes).to(device) # Mengganti layer .fc untuk ResNet dan pindahkan ke device
#     model_asli_gan_ratio_resnet.fc = nn.Sequential(
#         nn.Dropout(0.5), # Probabilitas Dropout, bisa disesuaikan
#         nn.Linear(model_asli_gan_ratio_resnet.fc.in_features, num_classes)
#     ).to(device)
#     model_asli_gan_ratio_resnet, history_asli_gan_ratio_resnet = train_model(
#         model=model_asli_gan_ratio_resnet,
#         train_loader=train_loader_asli_gan_ratio,
#         val_loader=val_loader_asli_gan_ratio,     # Menggunakan val_loader_asli_gan_ratio (sesuai instruksi Anda)
#         early_stopping=early_stopping_instance_asli_gan_ratio, # Gunakan instance baru
#         weight_decay=weight_decay,
#         best_model_path=f"best_model_asli_gan_ratio_resnet_{patience_name_suffix}.pth"
#     )

#     plot_metrics(history_asli_gan_ratio_resnet)
#     y_pred_asli_gan_ratio_resnet, y_true_asli_gan_ratio_resnet, mispredicted_data_asli_gan_ratio_resnet = evaluate_model(model_asli_gan_ratio_resnet, test_loader_asli_gan_ratio) # Menggunakan test_loader_asli_gan_ratio (sesuai instruksi Anda)
#     plot_confusion_matrix(y_true_asli_gan_ratio_resnet, y_pred_asli_gan_ratio_resnet, class_names)
#     plot_classification_metrics(y_true_asli_gan_ratio_resnet, y_pred_asli_gan_ratio_resnet, class_names)
#     plot_mispredicted_images(mispredicted_data_asli_gan_ratio_resnet, class_names, num_images_to_show=num_images_to_show)

# all_experiment_results_resnet = {}

# early_stopping_patience_configs = {
#     "es5": EARLY_STOPPING_PATIENCE_5,
#     "es10": EARLY_STOPPING_PATIENCE_10,
#     "es15": EARLY_STOPPING_PATIENCE_15,
#     # "es20": EARLY_STOPPING_PATIENCE_20,
#     "esNone": EARLY_STOPPING_PATIENCE_NONE
# }

# for suffix, patience_val in early_stopping_patience_configs.items():
#     all_experiment_results_resnet[suffix] = run_resnet_experiment_automated_naming(patience_val, suffix,
#                                                                                  train_loader_asli, val_loader_asli, test_loader_asli,
#                                                                                  train_loader_asli_gan, val_loader_asli_gan, test_loader_asli_gan,
#                                                                                  train_loader_asli_gan_ratio, val_loader_asli_gan_ratio, test_loader_asli_gan_ratio,
#                                                                                  class_names, NUM_CLASSES, DEVICE, WEIGHT_DECAY)


In [ ]:
# def run_pretrained_resnet_experiment_automated_naming(patience_value, patience_name_suffix,
#                                                    # DataLoaders sesuai penamaan Anda
#                                                    train_loader_asli_pretrained_resnet, val_loader_asli_pretrained_resnet, test_loader_asli_pretrained_resnet,
#                                                    train_loader_asli_gan_pretrained_resnet, val_loader_asli_gan_pretrained_resnet, test_loader_asli_gan_pretrained_resnet,
#                                                    train_loader_asli_gan_ratio_pretrained_resnet, val_loader_asli_gan_ratio_pretrained_resnet, test_loader_asli_gan_ratio_pretrained_resnet,
#                                                    class_names, num_classes, device, weight_decay, num_images_to_show=37):
#     """
#     Menjalankan eksperimen training dan evaluasi untuk ResNet50 (Pretrained) dengan penamaan variabel otomatis
#     berdasarkan konfigurasi early stopping.

#     Args:
#         patience_value: Nilai patience untuk EarlyStopping, atau None untuk menonaktifkan.
#         patience_name_suffix: String suffix untuk penamaan variabel (misal: "es5", "esNone").
#         ... (parameter DataLoaders sesuai nama yang Anda berikan) ...
#         class_names: List nama kelas.
#         num_classes: Jumlah kelas.
#         device: Device (cuda/cpu) untuk training.
#         weight_decay: Weight decay untuk optimizer.
#         num_images_to_show: Jumlah gambar mispredicted yang akan ditampilkan.
#     """

#     print(f'\n---------------------------Pretrained ResNet50 (TIMM) (Early Stopping {patience_name_suffix})---------------------------')

#     # --- Case: Data Asli ---
#     print(f'---------------------------Data Asli (Pretrained ResNet50 - ES {patience_name_suffix})---------------------------')
#     # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
#     early_stopping_instance_asli = None
#     if patience_value is not None:
#         early_stopping_instance_asli = EarlyStopping(patience=patience_value, verbose=True)

#     model_asli_pretrained_resnet = timm.create_model('resnet50', pretrained=True) # Menggunakan pretrained=True
#     # model_asli_pretrained_resnet.fc = nn.Linear(model_asli_pretrained_resnet.fc.in_features, num_classes).to(device) # Mengganti layer .fc untuk ResNet dan pindahkan ke device
#     model_asli_pretrained_resnet.fc = nn.Sequential(
#         nn.Dropout(0.5), # Probabilitas Dropout, bisa disesuaikan
#         nn.Linear(model_asli_pretrained_resnet.fc.in_features, num_classes)
#     ).to(device) # Pindahkan seluruh Sequential ke device

#     best_model_path_asli = f"best_model_asli_pretrained_resnet_{patience_name_suffix}.pth"

#     model_asli_pretrained_resnet, history_asli_pretrained_resnet = train_model(
#         model=model_asli_pretrained_resnet,
#         train_loader=train_loader_asli_pretrained_resnet,
#         val_loader=val_loader_asli_pretrained_resnet,
#         early_stopping=early_stopping_instance_asli, # Gunakan instance baru
#         weight_decay=weight_decay,
#         best_model_path=best_model_path_asli
#     )

#     plot_metrics(history_asli_pretrained_resnet)
#     y_pred_asli_pretrained_resnet, y_true_asli_pretrained_resnet, mispredicted_data_asli_pretrained_resnet = evaluate_model(model_asli_pretrained_resnet, test_loader_asli_pretrained_resnet)
#     plot_confusion_matrix(y_true_asli_pretrained_resnet, y_pred_asli_pretrained_resnet, class_names)
#     plot_classification_metrics(y_true_asli_pretrained_resnet, y_pred_asli_pretrained_resnet, class_names)
#     plot_mispredicted_images(mispredicted_data_asli_pretrained_resnet, class_names, num_images_to_show=num_images_to_show)

#     # --- Case: Data Asli + GAN ---
#     print(f'\n---------------------------Data Asli + GAN (Pretrained ResNet50 - ES {patience_name_suffix})---------------------------')
#     # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
#     early_stopping_instance_asli_gan = None
#     if patience_value is not None:
#         early_stopping_instance_asli_gan = EarlyStopping(patience=patience_value, verbose=True)

#     model_asli_gan_pretrained_resnet = timm.create_model('resnet50', pretrained=True)
#     # model_asli_gan_pretrained_resnet.fc = nn.Linear(model_asli_gan_pretrained_resnet.fc.in_features, num_classes).to(device)
#     model_asli_gan_pretrained_resnet.fc = nn.Sequential(
#         nn.Dropout(0.5), # Probabilitas Dropout, bisa disesuaikan
#         nn.Linear(model_asli_gan_pretrained_resnet.fc.in_features, num_classes)
#     ).to(device)
#     model_asli_gan_pretrained_resnet, history_asli_gan_pretrained_resnet = train_model(
#         model=model_asli_gan_pretrained_resnet,
#         train_loader=train_loader_asli_gan_pretrained_resnet,
#         val_loader=val_loader_asli_gan_pretrained_resnet, # Menggunakan val_loader_asli_gan_pretrained_resnet
#         early_stopping=early_stopping_instance_asli_gan, # Gunakan instance baru
#         weight_decay=weight_decay,
#         best_model_path=f"best_model_asli_gan_pretrained_resnet_{patience_name_suffix}.pth"
#     )

#     plot_metrics(history_asli_gan_pretrained_resnet)
#     y_pred_asli_gan_pretrained_resnet, y_true_asli_gan_pretrained_resnet, mispredicted_data_asli_gan_pretrained_resnet = evaluate_model(model_asli_gan_pretrained_resnet, test_loader_asli_gan_pretrained_resnet) # Menggunakan test_loader_asli_gan_pretrained_resnet
#     plot_confusion_matrix(y_true_asli_gan_pretrained_resnet, y_pred_asli_gan_pretrained_resnet, class_names)
#     plot_classification_metrics(y_true_asli_gan_pretrained_resnet, y_pred_asli_gan_pretrained_resnet, class_names)
#     plot_mispredicted_images(mispredicted_data_asli_gan_pretrained_resnet, class_names, num_images_to_show=num_images_to_show)

#     # --- Case: Data Asli + GAN + Ratio ---
#     print(f'\n---------------------------Data Asli + GAN + Ratio (Pretrained ResNet50 - ES {patience_name_suffix})---------------------------')
#     # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
#     early_stopping_instance_asli_gan_ratio = None
#     if patience_value is not None:
#         early_stopping_instance_asli_gan_ratio = EarlyStopping(patience=patience_value, verbose=True)

#     model_asli_gan_ratio_pretrained_resnet = timm.create_model('resnet50', pretrained=True)
#     # model_asli_gan_ratio_pretrained_resnet.fc = nn.Linear(model_asli_gan_ratio_pretrained_resnet.fc.in_features, num_classes).to(device)
#     model_asli_gan_ratio_pretrained_resnet.fc = nn.Sequential(
#         nn.Dropout(0.5), # Probabilitas Dropout, bisa disesuaikan
#         nn.Linear(model_asli_gan_ratio_pretrained_resnet.fc.in_features, num_classes)
#     ).to(device)

#     model_asli_gan_ratio_pretrained_resnet, history_asli_gan_ratio_pretrained_resnet = train_model(
#         model=model_asli_gan_ratio_pretrained_resnet,
#         train_loader=train_loader_asli_gan_ratio_pretrained_resnet,
#         val_loader=val_loader_asli_gan_ratio_pretrained_resnet, # Menggunakan val_loader_asli_gan_ratio_pretrained_resnet
#         early_stopping=early_stopping_instance_asli_gan_ratio, # Gunakan instance baru
#         weight_decay=weight_decay,
#         best_model_path=f"best_model_asli_gan_ratio_pretrained_resnet_{patience_name_suffix}.pth"
#     )

#     plot_metrics(history_asli_gan_ratio_pretrained_resnet)
#     y_pred_asli_gan_ratio_pretrained_resnet, y_true_asli_gan_ratio_pretrained_resnet, mispredicted_data_asli_gan_ratio_pretrained_resnet = evaluate_model(model_asli_gan_ratio_pretrained_resnet, test_loader_asli_gan_ratio_pretrained_resnet)
#     plot_confusion_matrix(y_true_asli_gan_ratio_pretrained_resnet, y_pred_asli_gan_ratio_pretrained_resnet, class_names)
#     plot_classification_metrics(y_true_asli_gan_ratio_pretrained_resnet, y_pred_asli_gan_ratio_pretrained_resnet, class_names)
#     plot_mispredicted_images(mispredicted_data_asli_gan_ratio_pretrained_resnet, class_names, num_images_to_show=num_images_to_show)

# all_experiment_results_pretrained_resnet = {}

# early_stopping_patience_configs = {
#     "es5": EARLY_STOPPING_PATIENCE_5,
#     "es10": EARLY_STOPPING_PATIENCE_10,
#     "es15": EARLY_STOPPING_PATIENCE_15,
#     # "es20": EARLY_STOPPING_PATIENCE_20,
#     "esNone": EARLY_STOPPING_PATIENCE_NONE
# }

# for suffix, patience_val in early_stopping_patience_configs.items():
#     all_experiment_results_pretrained_resnet[suffix] = run_pretrained_resnet_experiment_automated_naming(patience_val, suffix,
#                                                                                  train_loader_asli_pretrained_resnet, val_loader_asli_pretrained_resnet, test_loader_asli_pretrained_resnet,
#                                                                                  train_loader_asli_gan_pretrained_resnet, val_loader_asli_gan_pretrained_resnet, test_loader_asli_gan_pretrained_resnet,
#                                                                                  train_loader_asli_gan_ratio_pretrained_resnet, val_loader_asli_gan_ratio_pretrained_resnet, test_loader_asli_gan_ratio_pretrained_resnet,
#                                                                                  class_names, NUM_CLASSES, DEVICE, WEIGHT_DECAY)

In [ ]:
# def run_effnet_experiment_automated_naming(patience_value, patience_name_suffix,
#                                           # DataLoaders sesuai penamaan Anda
#                                           train_loader_asli, val_loader_asli, test_loader_asli,
#                                           train_loader_asli_gan, val_loader_asli_gan, test_loader_asli_gan,
#                                           train_loader_asli_gan_ratio, val_loader_asli_gan_ratio, test_loader_asli_gan_ratio,
#                                           class_names, num_classes, device, weight_decay, num_images_to_show=37):
#     """
#     Menjalankan eksperimen training dan evaluasi untuk EfficientNetV2-B0 dengan penamaan variabel otomatis
#     berdasarkan konfigurasi early stopping.

#     Args:
#         patience_value: Nilai patience untuk EarlyStopping, atau None untuk menonaktifkan.
#         patience_name_suffix: String suffix untuk penamaan variabel (misal: "es5", "esNone").
#         ... (parameter DataLoaders sesuai nama yang Anda berikan) ...
#         class_names: List nama kelas.
#         num_classes: Jumlah kelas.
#         device: Device (cuda/cpu) untuk training.
#         weight_decay: Weight decay untuk optimizer.
#         num_images_to_show: Jumlah gambar mispredicted yang akan ditampilkan.
#     """

#     print(f'\n---------------------------EfficientNetV2-B0 (TIMM) (Early Stopping {patience_name_suffix})---------------------------')

#     # --- Case: Data Asli ---
#     print(f'---------------------------Data Asli (EfficientNetV2-B0 - ES {patience_name_suffix})---------------------------')
#     # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
#     early_stopping_instance_asli = None
#     if patience_value is not None:
#         early_stopping_instance_asli = EarlyStopping(patience=patience_value, verbose=True)

#     model_asli_effnet = timm.create_model('tf_efficientnetv2_s.in21k_ft_in1k', pretrained=False) # Menggunakan pretrained=False sesuai kode Anda
#     # model_asli_effnet.classifier = nn.Linear(model_asli_effnet.classifier.in_features, num_classes).to(device) # Mengganti layer .classifier dan pindahkan ke device
#     model_asli_effnet.classifier = nn.Sequential(
#         nn.Dropout(0.5), # Probabilitas Dropout, bisa disesuaikan
#         nn.Linear(model_asli_effnet.classifier.in_features, num_classes)
#     ).to(device) # Pindahkan seluruh Sequential ke device
#     best_model_path_asli = f"best_model_asli_effnet_{patience_name_suffix}.pth"

#     model_asli_effnet, history_asli_effnet = train_model(
#         model=model_asli_effnet,
#         train_loader=train_loader_asli,
#         val_loader=val_loader_asli,
#         early_stopping=early_stopping_instance_asli, # Gunakan instance baru
#         weight_decay=weight_decay,
#         best_model_path=best_model_path_asli
#     )

#     plot_metrics(history_asli_effnet)
#     y_pred_asli_effnet, y_true_asli_effnet, mispredicted_data_asli_effnet = evaluate_model(model_asli_effnet, test_loader_asli)
#     plot_confusion_matrix(y_true_asli_effnet, y_pred_asli_effnet, class_names)
#     plot_classification_metrics(y_true_asli_effnet, y_pred_asli_effnet, class_names)
#     plot_mispredicted_images(mispredicted_data_asli_effnet, class_names, num_images_to_show=num_images_to_show)

#     # --- Case: Data Asli + GAN ---
#     print(f'\n---------------------------Data Asli + GAN (EfficientNetV2-B0 - ES {patience_name_suffix})---------------------------')
#     # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
#     early_stopping_instance_asli_gan = None
#     if patience_value is not None:
#         early_stopping_instance_asli_gan = EarlyStopping(patience=patience_value, verbose=True)

#     model_asli_gan_effnet = timm.create_model('tf_efficientnetv2_s.in21k_ft_in1k', pretrained=False) # Menggunakan pretrained=False sesuai kode Anda
#     # model_asli_gan_effnet.classifier = nn.Linear(model_asli_gan_effnet.classifier.in_features, num_classes).to(device)
#     model_asli_gan_effnet.classifier = nn.Sequential(
#         nn.Dropout(0.5), # Probabilitas Dropout, bisa disesuaikan
#         nn.Linear(model_asli_gan_effnet.classifier.in_features, num_classes)
#     ).to(device)

#     model_asli_gan_effnet, history_asli_gan_effnet = train_model(
#         model=model_asli_gan_effnet,
#         train_loader=train_loader_asli_gan,
#         val_loader=val_loader_asli_gan, # Menggunakan val_loader_asli_gan (sesuai instruksi Anda)
#         early_stopping=early_stopping_instance_asli_gan, # Gunakan instance baru
#         weight_decay=weight_decay,
#         best_model_path=f"best_model_asli_gan_effnet_{patience_name_suffix}.pth"
#     )

#     plot_metrics(history_asli_gan_effnet)
#     y_pred_asli_gan_effnet, y_true_asli_gan_effnet, mispredicted_data_asli_gan_effnet = evaluate_model(model_asli_gan_effnet, test_loader_asli_gan) # Menggunakan test_loader_asli_gan (sesuai instruksi Anda)
#     plot_confusion_matrix(y_true_asli_gan_effnet, y_pred_asli_gan_effnet, class_names)
#     plot_classification_metrics(y_true_asli_gan_effnet, y_pred_asli_gan_effnet, class_names)
#     plot_mispredicted_images(mispredicted_data_asli_gan_effnet, class_names, num_images_to_show=num_images_to_show)

#     # --- Case: Data Asli + GAN + Ratio ---
#     print(f'\n---------------------------Data Asli + GAN + Ratio (EfficientNetV2-B0 - ES {patience_name_suffix})---------------------------')
#     # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
#     early_stopping_instance_asli_gan_ratio = None
#     if patience_value is not None:
#         early_stopping_instance_asli_gan_ratio = EarlyStopping(patience=patience_value, verbose=True)

#     model_asli_gan_ratio_effnet = timm.create_model('tf_efficientnetv2_s.in21k_ft_in1k', pretrained=False) # Menggunakan pretrained=False sesuai kode Anda
#     # model_asli_gan_ratio_effnet.classifier = nn.Linear(model_asli_gan_ratio_effnet.classifier.in_features, num_classes).to(device)
#     model_asli_gan_ratio_effnet.classifier = nn.Sequential(
#         nn.Dropout(0.5), # Probabilitas Dropout, bisa disesuaikan
#         nn.Linear(model_asli_gan_ratio_effnet.classifier.in_features, num_classes)
#     ).to(device)

#     model_asli_gan_ratio_effnet, history_asli_gan_ratio_effnet = train_model(
#         model=model_asli_gan_ratio_effnet,
#         train_loader=train_loader_asli_gan_ratio,
#         val_loader=val_loader_asli_gan_ratio,     # Menggunakan val_loader_asli_gan_ratio (sesuai instruksi Anda)
#         early_stopping=early_stopping_instance_asli_gan_ratio, # Gunakan instance baru
#         weight_decay=weight_decay,
#         best_model_path=f"best_model_asli_gan_ratio_effnet_{patience_name_suffix}.pth"
#     )

#     plot_metrics(history_asli_gan_ratio_effnet)
#     y_pred_asli_gan_ratio_effnet, y_true_asli_gan_ratio_effnet, mispredicted_data_asli_gan_ratio_effnet = evaluate_model(model_asli_gan_ratio_effnet, test_loader_asli_gan_ratio) # Menggunakan test_loader_asli_gan_ratio (sesuai instruksi Anda)
#     plot_confusion_matrix(y_true_asli_gan_ratio_effnet, y_pred_asli_gan_ratio_effnet, class_names)
#     plot_classification_metrics(y_true_asli_gan_ratio_effnet, y_pred_asli_gan_ratio_effnet, class_names)
#     plot_mispredicted_images(mispredicted_data_asli_gan_ratio_effnet, class_names, num_images_to_show=num_images_to_show)

# all_experiment_results_effnet={}

# early_stopping_patience_configs = {
#     "es5": EARLY_STOPPING_PATIENCE_5,
#     "es10": EARLY_STOPPING_PATIENCE_10,
#     "es15": EARLY_STOPPING_PATIENCE_15,
#     # "es20": EARLY_STOPPING_PATIENCE_20,
#     "esNone": EARLY_STOPPING_PATIENCE_NONE
# }

# for suffix, patience_val in early_stopping_patience_configs.items():
#     all_experiment_results_effnet[suffix] = run_effnet_experiment_automated_naming(patience_val, suffix,
#                                                                                 train_loader_asli, val_loader_asli, test_loader_asli,
#                                                                                 train_loader_asli_gan, val_loader_asli_gan, test_loader_asli_gan,
#                                                                                 train_loader_asli_gan_ratio, val_loader_asli_gan_ratio, test_loader_asli_gan_ratio,
#                                                                                 class_names, NUM_CLASSES, DEVICE, WEIGHT_DECAY)


In [ ]:
# def run_pretrained_effnet_experiment_automated_naming(patience_value, patience_name_suffix,
#                                                       # DataLoaders sesuai penamaan Anda (dengan _pretrained_efficientnet)
#                                                       train_loader_asli_pretrained_efficientnet, val_loader_asli_pretrained_efficientnet, test_loader_asli_pretrained_efficientnet,
#                                                       train_loader_asli_gan_pretrained_efficientnet, val_loader_asli_gan_pretrained_efficientnet, test_loader_asli_gan_pretrained_efficientnet,
#                                                       train_loader_asli_gan_ratio_pretrained_efficientnet, val_loader_asli_gan_ratio_pretrained_efficientnet, test_loader_asli_gan_ratio_pretrained_efficientnet,
#                                                       class_names, num_classes, device, weight_decay, num_images_to_show=37):
#     """
#     Menjalankan eksperimen training dan evaluasi untuk EfficientNetV2-B0 (Pretrained) dengan penamaan variabel otomatis
#     berdasarkan konfigurasi early stopping.

#     Args:
#         patience_value: Nilai patience untuk EarlyStopping, atau None untuk menonaktifkan.
#         patience_name_suffix: String suffix untuk penamaan variabel (misal: "es5", "esNone").
#         ... (parameter DataLoaders sesuai nama yang Anda berikan) ...
#         class_names: List nama kelas.
#         num_classes: Jumlah kelas.
#         device: Device (cuda/cpu) untuk training.
#         weight_decay: Weight decay untuk optimizer.
#         num_images_to_show: Jumlah gambar mispredicted yang akan ditampilkan.
#     """

#     print(f'\n---------------------------Pretrained EfficientNetV2-B0 (TIMM) (Early Stopping {patience_name_suffix})---------------------------')

#     # --- Case: Data Asli ---
#     print(f'---------------------------Data Asli (Pretrained EfficientNetV2-B0 - ES {patience_name_suffix})---------------------------')
#     # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
#     early_stopping_instance_asli = None
#     if patience_value is not None:
#         early_stopping_instance_asli = EarlyStopping(patience=patience_value, verbose=True)

#     model_asli_pretrained_effnet = timm.create_model('tf_efficientnetv2_s.in21k_ft_in1k', pretrained=True) # Menggunakan pretrained=True
#     # model_asli_pretrained_effnet.classifier = nn.Linear(model_asli_pretrained_effnet.classifier.in_features, num_classes).to(device) # Mengganti layer .classifier dan pindahkan ke device
#     model_asli_pretrained_effnet.classifier = nn.Sequential(
#         nn.Dropout(0.5), # Probabilitas Dropout, bisa disesuaikan
#         nn.Linear(model_asli_pretrained_effnet.classifier.in_features, num_classes)
#     ).to(device) # Pindahkan seluruh Sequential ke device

#     best_model_path_asli = f"best_model_asli_pretrained_effnet_{patience_name_suffix}.pth"

#     model_asli_pretrained_effnet, history_asli_effnet_pretrained = train_model(
#         model=model_asli_pretrained_effnet,
#         train_loader=train_loader_asli_pretrained_efficientnet,
#         val_loader=val_loader_asli_pretrained_efficientnet,
#         early_stopping=early_stopping_instance_asli, # Gunakan instance baru
#         weight_decay=weight_decay,
#         best_model_path=best_model_path_asli
#     )

#     plot_metrics(history_asli_effnet_pretrained)
#     y_pred_asli_effnet_pretrained, y_true_asli_effnet_pretrained, mispredicted_data_asli_effnet_pretrained = evaluate_model(model_asli_pretrained_effnet, test_loader_asli_pretrained_efficientnet)
#     plot_confusion_matrix(y_true_asli_effnet_pretrained, y_pred_asli_effnet_pretrained, class_names)
#     plot_classification_metrics(y_true_asli_effnet_pretrained, y_pred_asli_effnet_pretrained, class_names)
#     plot_mispredicted_images(mispredicted_data_asli_effnet_pretrained, class_names, num_images_to_show=num_images_to_show)

#     # --- Case: Data Asli + GAN ---
#     print(f'\n---------------------------Data Asli + GAN (Pretrained EfficientNetV2-B0 - ES {patience_name_suffix})---------------------------')
#     # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
#     early_stopping_instance_asli_gan = None
#     if patience_value is not None:
#         early_stopping_instance_asli_gan = EarlyStopping(patience=patience_value, verbose=True)

#     model_asli_gan_pretrained_effnet = timm.create_model('tf_efficientnetv2_s.in21k_ft_in1k', pretrained=True)
#     # model_asli_gan_pretrained_effnet.classifier = nn.Linear(model_asli_gan_pretrained_effnet.classifier.in_features, num_classes).to(device)
#     model_asli_gan_pretrained_effnet.classifier = nn.Sequential(
#         nn.Dropout(0.5), # Probabilitas Dropout, bisa disesuaikan
#         nn.Linear(model_asli_gan_pretrained_effnet.classifier.in_features, num_classes)
#     ).to(device)

#     model_asli_gan_pretrained_effnet, history_asli_gan_effnet_pretrained = train_model(
#         model=model_asli_gan_pretrained_effnet,
#         train_loader=train_loader_asli_gan_pretrained_efficientnet,
#         val_loader=val_loader_asli_gan_pretrained_efficientnet, # Menggunakan val_loader_asli_gan_pretrained_efficientnet
#         early_stopping=early_stopping_instance_asli_gan, # Gunakan instance baru
#         weight_decay=weight_decay,
#         best_model_path=f"best_model_asli_gan_pretrained_effnet_{patience_name_suffix}.pth"
#     )

#     plot_metrics(history_asli_gan_effnet_pretrained)
#     y_pred_asli_gan_effnet_pretrained, y_true_asli_gan_effnet_pretrained, mispredicted_data_asli_gan_effnet_pretrained = evaluate_model(model_asli_gan_pretrained_effnet, test_loader_asli_gan_pretrained_efficientnet) # Menggunakan test_loader_asli_gan_pretrained_efficientnet
#     plot_confusion_matrix(y_true_asli_gan_effnet_pretrained, y_pred_asli_gan_effnet_pretrained, class_names)
#     plot_classification_metrics(y_true_asli_gan_effnet_pretrained, y_pred_asli_gan_effnet_pretrained, class_names)
#     plot_mispredicted_images(mispredicted_data_asli_gan_effnet_pretrained, class_names, num_images_to_show=num_images_to_show)

#     # --- Case: Data Asli + GAN + Ratio ---
#     print(f'\n---------------------------Data Asli + GAN + Ratio (Pretrained EfficientNetV2-B0 - ES {patience_name_suffix})---------------------------')
#     # Inisialisasi early_stopping_instance DI SINI untuk setiap case dataset
#     early_stopping_instance_asli_gan_ratio = None
#     if patience_value is not None:
#         early_stopping_instance_asli_gan_ratio = EarlyStopping(patience=patience_value, verbose=True)

#     model_asli_gan_ratio_pretrained_effnet = timm.create_model('tf_efficientnetv2_s.in21k_ft_in1k', pretrained=True)
#     # model_asli_gan_ratio_pretrained_effnet.classifier = nn.Linear(model_asli_gan_ratio_pretrained_effnet.classifier.in_features, num_classes).to(device)
#     model_asli_gan_ratio_pretrained_effnet.classifier = nn.Sequential(
#         nn.Dropout(0.5), # Probabilitas Dropout, bisa disesuaikan
#         nn.Linear(model_asli_gan_ratio_pretrained_effnet.classifier.in_features, num_classes)
#     ).to(device)

#     model_asli_gan_ratio_pretrained_effnet, history_asli_gan_ratio_effnet_pretrained = train_model(
#         model=model_asli_gan_ratio_pretrained_effnet,
#         train_loader=train_loader_asli_gan_ratio_pretrained_efficientnet,
#         val_loader=val_loader_asli_gan_ratio_pretrained_efficientnet, # Menggunakan val_loader_asli_gan_ratio_pretrained_efficientnet
#         early_stopping=early_stopping_instance_asli_gan_ratio, # Gunakan instance baru
#         weight_decay=weight_decay,
#         best_model_path=f"best_model_asli_gan_ratio_pretrained_effnet_{patience_name_suffix}.pth"
#     )

#     plot_metrics(history_asli_gan_ratio_effnet_pretrained)
#     y_pred_asli_gan_ratio_effnet_pretrained, y_true_asli_gan_ratio_effnet_pretrained, mispredicted_data_asli_gan_ratio_effnet_pretrained = evaluate_model(model_asli_gan_ratio_pretrained_effnet, test_loader_asli_gan_ratio_pretrained_efficientnet)
#     plot_confusion_matrix(y_true_asli_gan_ratio_effnet_pretrained, y_pred_asli_gan_ratio_effnet_pretrained, class_names)
#     plot_classification_metrics(y_true_asli_gan_ratio_effnet_pretrained, y_pred_asli_gan_ratio_effnet_pretrained, class_names)
#     plot_mispredicted_images(mispredicted_data_asli_gan_ratio_effnet_pretrained, class_names, num_images_to_show=num_images_to_show)

# all_experiment_results_pretrained_effnet = {}

# early_stopping_patience_configs = {
#     "es5": EARLY_STOPPING_PATIENCE_5,
#     "es10": EARLY_STOPPING_PATIENCE_10,
#     "es15": EARLY_STOPPING_PATIENCE_15,
#     # "es20": EARLY_STOPPING_PATIENCE_20,
#     "esNone": EARLY_STOPPING_PATIENCE_NONE
# }

# for suffix, patience_val in early_stopping_patience_configs.items():
#     all_experiment_results_pretrained_effnet[suffix] = run_pretrained_effnet_experiment_automated_naming(patience_val, suffix,
#                                                                                  train_loader_asli_pretrained_efficientnet, val_loader_asli_pretrained_efficientnet, test_loader_asli_pretrained_efficientnet,
#                                                                                  train_loader_asli_gan_pretrained_efficientnet, val_loader_asli_gan_pretrained_efficientnet, test_loader_asli_gan_pretrained_efficientnet,
#                                                                                  train_loader_asli_gan_ratio_pretrained_efficientnet, val_loader_asli_gan_ratio_pretrained_efficientnet, test_loader_asli_gan_ratio_pretrained_efficientnet,
#                                                                                  class_names, NUM_CLASSES, DEVICE, WEIGHT_DECAY)
